In [1]:
from bs4 import BeautifulSoup



def extract_text_from_html(file_path):

    with open(file_path, 'r', encoding='utf-8') as file:

        html_content = file.read()

    soup = BeautifulSoup(html_content, 'html.parser')

    paragraphs = soup.find_all('p')

    text_content = "\n".join([p.get_text() for p in paragraphs])

    return text_content



# File paths for the different laws

file_paths = {

    'gdpr': '/kaggle/input/spanish-laws/spanish_gdpr.html',

    'ai_act': '/kaggle/input/spanish-laws/spanish_AI_act.html',

    'dma': '/kaggle/input/spanish-laws/spanish_dma.html',

    'dsa': '/kaggle/input/spanish-laws/spanish_dsa.html'

}



# Extract text for each law

texts = {law: extract_text_from_html(path) for law, path in file_paths.items()}



# Example usage to print the first 1000 characters of each law's text

for law, text in texts.items():

    print(f"First 1000 characters of {law.upper()}:\n{text[:1000]}\n")


First 1000 characters of GDPR:
4.5.2016   
ES
Diario Oficial de la Unión Europea
L 119/1
REGLAMENTO (UE) 2016/679 DEL PARLAMENTO EUROPEO Y DEL CONSEJO
de 27 de abril de 2016
relativo a la protección de las personas físicas en lo que respecta al tratamiento de datos personales y a la libre circulación de estos datos y por el que se deroga la Directiva 95/46/CE (Reglamento general de protección de datos)
(Texto pertinente a efectos del EEE)
EL PARLAMENTO EUROPEO Y EL CONSEJO DE LA UNIÓN EUROPEA,
Visto el Tratado de Funcionamiento de la Unión Europea, y en particular su artículo 16,
Vista la propuesta de la Comisión Europea,
Previa transmisión del proyecto de texto legislativo a los Parlamentos nacionales,
Visto el dictamen del Comité Económico y Social Europeo (1),
Visto el dictamen del Comité de las Regiones (2),
De conformidad con el procedimiento legislativo ordinario (3),
Considerando lo siguiente:
(1)
La protección de las personas físicas en relación con el tratamiento de datos pers

In [2]:
!pip install -U langchain-community

!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 19.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.0/307.0 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: requests-toolbelt
    Found existing installation: requests-toolbelt 0.10.1
    Uninstalling requests-toolbelt-0.10.1:
      Successfully uninstalled requests-toolbelt-0.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depend

In [3]:
!pip install chromadb

import chromadb

import nltk

from nltk.tokenize import sent_tokenize

from transformers import AutoTokenizer

from langchain.embeddings import HuggingFaceBgeEmbeddings

from bs4 import BeautifulSoup



nltk.download('punkt')



tokenizer = AutoTokenizer.from_pretrained('bigscience/bloomz')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [4]:
def chunk_text_based_on_tokens(text, max_tokens=300):

    sentences = sent_tokenize(text)

    chunks = []

    current_chunk = []

    current_length = 0



    for sentence in sentences:

        sentence_length = len(tokenizer.tokenize(sentence))

        if current_length + sentence_length <= max_tokens:

            current_chunk.append(sentence)

            current_length += sentence_length

        else:

            chunks.append(" ".join(current_chunk))

            current_chunk = [sentence]

            current_length = sentence_length



    if current_chunk:

        chunks.append(" ".join(current_chunk))



    return chunks



def extract_sections_articles_chapters(soup):

    sections = []

    current_section = []

    for element in soup.find_all(['h1', 'h2', 'h3', 'p']):

        if element.name in ['h1', 'h2', 'h3']:

            if current_section:

                sections.append(" ".join(current_section))

                current_section = []

            current_section.append(element.get_text())

        else:

            current_section.append(element.get_text())

    if current_section:

        sections.append(" ".join(current_section))

    return sections



def load_and_process_html(file_path):

    with open(file_path, 'r', encoding='utf-8') as file:

        html_content = file.read()

    soup = BeautifulSoup(html_content, 'html.parser')

    sections = extract_sections_articles_chapters(soup)

    all_chunks = []

    for section in sections:

        all_chunks.extend(chunk_text_based_on_tokens(section))

    return all_chunks



def embed_chunks(chunks, model_name):

    encode_kwargs = {'normalize_embeddings': True}

    model_norm = HuggingFaceBgeEmbeddings(

        model_name=model_name,

        model_kwargs={'device': 'cuda'},

        encode_kwargs=encode_kwargs

    )

    embeddings = model_norm.embed_documents(chunks)

    return embeddings



def setup_chroma_collection(client, collection_name):

    try:

        client.delete_collection(name=collection_name)

    except Exception as e:

        print(f"Error deleting collection: {e}")

    try:

        collection = client.create_collection(name=collection_name)

        return collection

    except Exception as e:

        print(f"Error creating collection: {e}")

        return None



def process_and_store_embeddings(file_path, collection_name, model_name, model_norm):

    chunks = load_and_process_html(file_path)

    embeddings = embed_chunks(chunks, model_name)



    chroma_client = chromadb.Client()

    collection = setup_chroma_collection(chroma_client, collection_name)



    for i, embedding in enumerate(embeddings):

        collection.add(

            documents=[chunks[i]],

            ids=[f"id_{i}"],

            embeddings=[embedding]

        )

    return collection, chunks, embeddings



def embed_and_query(query, model_norm, collection, top_k=10):

    query_embedding = embed_query(query, model_norm)

    results = query_chroma_db(query_embedding, collection, top_k)

    return results



def embed_query(query, model_name):

    query_embedding = model_name.embed_documents([query])

    return query_embedding[0]



def query_chroma_db(query_embedding, collection, top_k=10):

    results = collection.query(

        query_embeddings=[query_embedding],

        n_results=top_k

    )

    return results

In [5]:
# File paths and collection names for the different laws

laws_info = {

    'gdpr': {

        'file_path': '/kaggle/input/spanish-laws/spanish_gdpr.html',

        'collection_name': 'embeddings_gdpr',

        'query': "¿Cómo contribuye el reglamento mencionado en el texto a garantizar la libre circulación de datos personales en la Unión Europea, y qué consideraciones se tienen en cuenta para las microempresas y las pequeñas y medianas empresas?"

    },

    'ai_act': {

        'file_path': '/kaggle/input/spanish-laws/spanish_AI_act.html',

        'collection_name': 'embeddings_ai_act',

        'query': "¿Qué importancia tiene la capacidad de inferencia en la definición de un sistema de IA según el reglamento descrito, y cómo se diferencia esta capacidad de los enfoques de programación tradicionales?"

    },

    'dma': {

        'file_path': '/kaggle/input/spanish-laws/spanish_dma.html',

        'collection_name': 'embeddings_dma',

        'query': "¿Cómo asegura la Comisión Europea una participación equitativa del Parlamento Europeo y el Consejo en la preparación de los actos delegados relacionados con la modificación de los anexos I, II y IV del Reglamento sobre productos de doble uso?"

    },

    'dsa': {

        'file_path': '/kaggle/input/spanish-laws/spanish_dsa.html',

        'collection_name': 'embeddings_dsa',

        'query': "¿Cómo define el concepto de contenido ilícito el presente Reglamento, y qué ejemplos específicos se mencionan para ilustrar este concepto?"

    }

}



model_name = "dccuchile/bert-base-spanish-wwm-cased"

encode_kwargs = {'normalize_embeddings': True}



model_norm = HuggingFaceBgeEmbeddings(

    model_name=model_name,

    model_kwargs={'device': 'cuda'},

    encode_kwargs=encode_kwargs

)



# Process and store embeddings for each law

collections = {}

chunks_dict = {}

embeddings_dict = {}



for law, info in laws_info.items():

    print(f"Processing {law}...")

    collection, chunks, embeddings = process_and_store_embeddings(info['file_path'], info['collection_name'], model_name, model_norm)

    collections[law] = collection

    chunks_dict[law] = chunks

    embeddings_dict[law] = embeddings



# Utility function to print specific chunk or embedding

def print_specific_chunk_or_embedding(law, index, data_type='chunk'):

    if data_type == 'chunk':

        if law in chunks_dict and len(chunks_dict[law]) > index:

            print(f"{index + 1}th chunk of {law.upper()}:\n{chunks_dict[law][index]}\n")

        else:

            print(f"{law.upper()} does not have {index + 1} chunks or law not found.")

    elif data_type == 'embedding':

        if law in embeddings_dict and len(embeddings_dict[law]) > index:

            print(f"{index + 1}th embedding of {law.upper()}:\n{embeddings_dict[law][index]}\n")

        else:

            print(f"{law.upper()} does not have {index + 1} embeddings or law not found.")

    else:

        print("Invalid data type specified. Use 'chunk' or 'embedding'.")





# Query each law and print results

for law, info in laws_info.items():

    print(f"\nQuerying {law.upper()} collection:")

    results = embed_and_query(info['query'], model_norm, collections[law], top_k=1)



    # Ensure only one result is retrieved

    if results['documents']:

        retrieved_context = results['documents'][0]

        chunk_id = results['ids'][0][0]  # Accessing the first element in the list of IDs



        # Print the chunk number and law name

        print(f"Retrieved chunk {chunk_id.split('_')[-1]} from {law.upper()}:")

        print(retrieved_context)

    else:

        print(f"No results found for {law.upper()}.")




config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

...

Error deleting collection: Collection embeddings_ai_act does not exist.
Processing dma...


Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Error deleting collection: Collection embeddings_dma does not exist.
Processing dsa...


Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Error deleting collection: Collection embeddings_dsa does not exist.

Querying GDPR collection:
Retrieved chunk 6 from GDPR:
['(12) El artículo\xa016, apartado\xa02, del\xa0TFUE encomienda al Parlamento Europeo y al Consejo que establezcan las normas sobre protección de las personas físicas respecto del tratamiento de datos de carácter personal y las normas relativas a la libre circulación de dichos datos. (13) Para garantizar un nivel coherente de protección de las personas físicas en toda la Unión y evitar divergencias que dificulten la libre circulación de datos personales dentro del mercado interior, es necesario un reglamento que proporcione seguridad jurídica y transparencia a los operadores económicos, incluidas las microempresas y las pequeñas y medianas empresas, y ofrezca a las personas físicas de todos los Estados miembros el mismo nivel de derechos y obligaciones exigibles y de responsabilidades para los responsables y encargados del tratamiento, con el fin de garantizar un

In [10]:
print_specific_chunk_or_embedding('ai_act', 399, 'chunk')

400th chunk of AI_ACT:
Toda persona que se vea afectada por una decisión que el responsable del despliegue adopte basándose en los resultados de salida de un sistema de IA de alto riesgo que figure en el anexo III, con excepción de los sistemas enumerados en su punto 2, y que produzca efectos jurídicos o le afecte considerablemente del mismo modo, de manera que considere que tiene un efecto perjudicial para su salud, su seguridad o sus derechos fundamentales, tendrá derecho a obtener del responsable del despliegue explicaciones claras y significativas acerca del papel que el sistema de IA ha tenido en el proceso de toma de decisiones y los principales elementos de la decisión adoptada. 2. No se aplicará el apartado 1 a la utilización de sistemas de IA para los que existan excepciones o restricciones a la obligación prevista en dicho apartado derivadas del Derecho de la Unión o nacional de conformidad con el Derecho de la Unión. 3. El presente artículo se aplicará únicamente en la medid

In [6]:
from transformers import AutoTokenizer, AutoModel

from sentence_transformers import SentenceTransformer, util

import torch

import numpy as np

from sklearn.metrics.pairwise import cosine_similarity



# Load BETO model and tokenizer for cosine similarity

tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

model = AutoModel.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")



# Load LaBSE model for semantic similarity

semantic_model = SentenceTransformer('xlm-roberta-base')



# Function to generate embeddings using BERT for cosine similarity

def generate_bert_embedding(text, tokenizer, model):

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    with torch.no_grad():

        outputs = model(**inputs)

    embedding = outputs.last_hidden_state[:, 0, :].numpy()  # [CLS] token embedding

    return embedding



# Function to calculate cosine similarity

def calculate_cosine_similarity(reference_embedding, retrieved_embedding):

    return cosine_similarity(reference_embedding.reshape(1, -1), retrieved_embedding.reshape(1, -1))[0][0]



# Function to calculate semantic similarity using Sentence-Transformers

def calculate_semantic_similarity(reference_text, retrieved_text, model):

    embeddings1 = model.encode(reference_text, convert_to_tensor=True)

    embeddings2 = model.encode(retrieved_text, convert_to_tensor=True)

    similarity = util.pytorch_cos_sim(embeddings1, embeddings2)

    return similarity.item()



reference_answers = {

    'gdpr': "Para garantizar un nivel coherente de protección de las personas físicas en toda la Unión y evitar divergencias que dificulten la libre circulación de datos personales dentro del mercado interior, es necesario un reglamento que proporcione seguridad jurídica y transparencia a los operadores económicos, incluidas las microempresas y las pequeñas y medianas empresas, y ofrezca a las personas físicas de todos los Estados miembros el mismo nivel de derechos y obligaciones exigibles y de responsabilidades para los responsables y encargados del tratamiento, con el fin de garantizar una supervisión coherente del tratamiento de datos personales y sanciones equivalentes en todos los Estados miembros, así como la cooperación efectiva entre las autoridades de control de los diferentes Estados miembros. El buen funcionamiento del mercado interior exige que la libre circulación de los datos personales en la Unión no sea restringida ni prohibida por motivos relacionados con la protección de las personas físicas en lo que respecta al tratamiento de datos personales. Con objeto de tener en cuenta la situación específica de las microempresas y las pequeñas y medianas empresas, el presente Reglamento incluye una serie de excepciones en materia de llevanza de registros para organizaciones con menos de 250 empleados. Además, alienta a las instituciones y órganos de la Unión y a los Estados miembros y a sus autoridades de control a tener en cuenta las necesidades específicas de las microempresas y las pequeñas y medianas empresas en la aplicación del presente Reglamento. El concepto de microempresas y pequeñas y medianas empresas debe extraerse del artículo 2 del anexo de la Recomendación 2003/361/CE de la Comisión (5).",

    'ai_act': "Debe definirse con claridad el concepto de «sistema de IA» en el presente Reglamento y armonizarlo estrechamente con los trabajos de las organizaciones internacionales que se ocupan de la IA, a fin de garantizar la seguridad jurídica y facilitar la convergencia a escala internacional y una amplia aceptación, al mismo tiempo que se prevé la flexibilidad necesaria para dar cabida a los rápidos avances tecnológicos en este ámbito. Además, la definición debe basarse en las principales características de los sistemas de IA que los distinguen de los sistemas de software o los planteamientos de programación tradicionales y más sencillos, y no debe incluir los sistemas basados en las normas definidas únicamente por personas físicas para ejecutar automáticamente operaciones. Una característica principal de los sistemas de IA es su capacidad de inferencia. Esta capacidad de inferencia se refiere al proceso de obtención de resultados de salida, como predicciones, contenidos, recomendaciones o decisiones, que puede influir en entornos físicos y virtuales, y a la capacidad de los sistemas de IA para deducir modelos o algoritmos, o ambos, a partir de información de entrada o datos. Las técnicas que permiten la inferencia al construir un sistema de IA incluyen estrategias de aprendizaje automático que aprenden de los datos cómo alcanzar determinados objetivos y estrategias basadas en la lógica y el conocimiento que infieren a partir de conocimientos codificados o de una representación simbólica de la tarea que debe resolverse. La capacidad de inferencia de un sistema de IA trasciende el tratamiento básico de datos, al permitir el aprendizaje, el razonamiento o la modelización. El término «basado en una máquina» se refiere al hecho de que los sistemas de IA se ejecutan en máquinas.La referencia a objetivos explícitos o implícitos subraya que los sistemas de IA pueden funcionar con arreglo a objetivos definidos explícitos o a objetivos implícitos. Los objetivos del sistema de IA pueden ser diferentes de la finalidad prevista del sistema de IA en un contexto específico. A los efectos del presente Reglamento, debe entenderse por entornos los contextos en los que funcionan los sistemas de IA, mientras que los resultados de salida generados por el sistema de IA reflejan las distintas funciones desempeñadas por los sistemas de IA e incluyen predicciones, contenidos, recomendaciones o decisiones. Los sistemas de IA están diseñados para funcionar con distintos niveles de autonomía, lo que significa que pueden actuar con cierto grado de independencia con respecto a la actuación humana y tienen ciertas capacidades para funcionar sin intervención humana. La capacidad de adaptación que un sistema de IA podría mostrar tras su despliegue se refiere a las capacidades de autoaprendizaje que permiten al sistema cambiar mientras está en uso. Los sistemas de IA pueden utilizarse de manera independiente o como componentes de un producto, con independencia de si el sistema forma parte físicamente del producto (integrado) o contribuye a la funcionalidad del producto sin formar parte de él (no integrado).",

    'dma': "A fin de permitir una rápida respuesta de la Unión a las circunstancias cambiantes en la evaluación de la sensibilidad de las exportaciones al amparo de las autorizaciones generales de exportación de la Unión, así como a la evolución tecnológica y comercial, deben delegarse en la Comisión los poderes para adoptar actos con arreglo al artículo 290 del Tratado de Funcionamiento de la Unión Europea (TFUE), por lo que respecta a modificar los anexos I, II y IV del presente Reglamento. Las decisiones de actualización de la lista común de productos de doble uso sujetos a controles de exportación establecida en el anexo I deben ajustarse a las obligaciones y compromisos que los Estados miembros o la Unión hayan asumido como partes en los acuerdos internacionales de no proliferación y participantes en los regímenes multilaterales de control de las exportaciones pertinentes o en virtud de la ratificación de tratados internacionales aplicables. Cuando la modificación del anexo I afecte a productos de doble uso que también están incluidos en el anexo I o IV, dichos anexos deben modificarse en consecuencia. Las decisiones de actualización de las listas comunes de productos y destinos establecidas en las secciones A a H del anexo II deben tomarse teniendo en cuenta los criterios de evaluación establecidos en el presente Reglamento. Reviste especial importancia que la Comisión lleve a cabo las consultas oportunas durante la fase preparatoria, en particular con expertos, y que esas consultas se realicen de conformidad con los principios establecidos en el Acuerdo interinstitucional de 13 de abril de 2016 sobre la mejora de la legislación (5). En particular, a fin de garantizar una participación equitativa en la preparación de los actos delegados, el Parlamento Europeo y el Consejo reciben toda la documentación al mismo tiempo que los expertos de los Estados miembros, y sus expertos tienen acceso sistemáticamente a las reuniones de los grupos de expertos de la Comisión que se ocupen de la preparación de actos delegados.",

    'dsa': "A fin de alcanzar el objetivo de garantizar un entorno en línea seguro, predecible y digno de confianza, para los efectos del presente Reglamento, el concepto de «contenido ilícito» debe reflejar a grandes rasgos las normas vigentes en el entorno fuera de línea. Concretamente, el concepto de «contenido ilícito» debe definirse de manera amplia para abarcar la información relacionada con contenidos, productos, servicios y actividades de carácter ilícito. En particular, debe entenderse que dicho concepto se refiere a información, sea cual sea su forma, que sea de por sí ilícita en virtud del Derecho aplicable, como los delitos de incitación al odio o los contenidos terroristas y los contenidos discriminatorios ilícitos, o que las normas aplicables consideren ilícita por estar relacionada con actividades ilícitas. Ejemplos de ello son el intercambio de imágenes que representen abusos sexuales de menores, el intercambio ilícito no consentido de imágenes privadas, el acoso en línea, la venta de productos no conformes o falsificados, la venta de productos o la prestación de servicios que infrinjan el Derecho en materia de protección de los consumidores, el uso no autorizado de material protegido por derechos de autor, la oferta ilegal de servicios de alojamiento o la venta ilegal de animales vivos. En cambio, el vídeo de un testigo presencial de un posible delito no debe considerarse contenido ilícito por el mero hecho de que muestre un acto ilícito, cuando la grabación o difusión pública de dicho vídeo no sea ilícita con arreglo al Derecho nacional o de la Unión. En este sentido, es irrelevante tanto que el carácter ilícito de la información o actividad se derive del Derecho de la Unión o del Derecho nacional que sea conforme con el Derecho de la Unión, como la naturaleza o materia precisa del Derecho aplicable."

}



# Calculate and print similarities

similarities = []



for law, info in laws_info.items():

    print(f"\nQuerying {law.upper()} collection:")

    results = embed_and_query(info['query'], model_norm, collections[law], top_k=1)



    if results and 'documents' in results and results['documents']:

        retrieved_text = results['documents'][0][0]  # Assuming it's a list of lists



        # Generate embeddings using BERT for cosine similarity

        retrieved_embedding = generate_bert_embedding(retrieved_text, tokenizer, model)

        reference_embedding = generate_bert_embedding(reference_answers[law], tokenizer, model)



        # Calculate cosine similarity using BERT embeddings

        cosine_sim = calculate_cosine_similarity(reference_embedding, retrieved_embedding)



        # Calculate semantic similarity using Sentence-Transformers model

        semantic_sim = calculate_semantic_similarity(reference_answers[law], retrieved_text, semantic_model)



        # Store the results

        similarities.append({

            'law': law,

            'retrieved_answer': retrieved_text,

            'cosine_similarity': cosine_sim,

            'semantic_similarity': semantic_sim

        })



        # Print the results for this law

        chunk_id = results['ids'][0][0]  # Accessing the first element in the list of IDs

        print(f"Retrieved chunk {chunk_id.split('_')[-1]} from {law.upper()}:")

        print(retrieved_text)

        print(f"Cosine Similarity with reference answer: {cosine_sim:.4f}")

        print(f"Semantic Similarity with reference answer: {semantic_sim:.4f}")

        print("----\n")

    else:

        print(f"No valid results found for {law.upper()} in the query.")

Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieved chunk 10 from DMA:
Las decisiones de actualización de la lista común de productos de doble uso sujetos a controles de exportación establecida en el anexo I deben ajustarse a las obligaciones y compromisos que los Estados miembros o la Unión hayan asumido como partes en los acuerdos internacionales de no proliferación y participantes en los regímenes multilaterales de control de las exportaciones pertinentes o en virtud de la ratificación de tratados internacionales aplicables. Cuando la modificación del anexo I afecte a productos de doble uso que también están incluidos en el anexo I o IV, dichos anexos deben modificarse en consecuencia. Las decisiones de actualización de las listas comunes de productos y destinos establecidas en las secciones A a H del anexo II deben tomarse teniendo en cuenta los criterios de evaluación establecidos en el presente Reglamento. Reviste especial importancia que la Comisión lleve a cabo las consultas oportunas durante la fase preparatoria, en p

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieved chunk 106 from DSA:
(106) Las disposiciones del presente Reglamento relativas a los códigos de conducta podrían servir de base para iniciativas de autorregulación ya establecidas a escala de la Unión, como el compromiso de seguridad con los productos, el memorando de entendimiento sobre la venta de productos falsificados a través de internet, el Código de conducta para la lucha contra la incitación ilegal al odio en internet, y el Código de buenas prácticas en materia de desinformación. En particular en relación con este último, a raíz de las orientaciones de la Comisión, el Código de buenas prácticas en materia de desinformación ha sido reforzado como se anunció en el Plan de Acción para la Democracia Europea. (107) En la publicidad en línea generalmente intervienen varios actores, incluidos los servicios intermediarios que conectan a los publicistas con los anunciantes. Los códigos de conducta deben apoyar y complementar las obligaciones de transparencia relativas a la publ

In [10]:
from collections import defaultdict

from transformers import pipeline


integrated_questions_answers = [

# Question 1 from GDPR

{

    'law': 'gdpr',

    'question': "¿Cuál es el derecho fundamental en relación con el tratamiento de datos personales según la Carta de los Derechos Fundamentales de la Unión Europea?",

    'answer': "La protección de las personas físicas en relación con el tratamiento de datos personales es un derecho fundamental. El artículo 8(1) de la Carta de los Derechos Fundamentales de la Unión Europea (‘la Carta’) y el artículo 16(1) del Tratado de Funcionamiento de la Unión Europea (TFUE) establecen que toda persona tiene derecho a la protección de los datos personales que le conciernen. Este Reglamento pretende contribuir a la realización de un área de libertad, seguridad y justicia y de una unión económica, al progreso económico y social, al fortalecimiento y la convergencia de las economías en el mercado interior, y al bienestar de las personas físicas."

},

# Question 1 from AI Act

{

    'law': 'ai_act',

    'question': "¿Cuáles son los principales objetivos de la Ley de IA en relación con el desarrollo y uso de la IA en la Unión Europea?",

    'answer': "La Ley de IA tiene como objetivo garantizar que los sistemas de IA comercializados y utilizados en la Unión sean seguros, respeten la legislación existente sobre derechos fundamentales y los valores de la Unión, y no menoscaben los derechos fundamentales. La Ley busca establecer un marco legal que aborde los riesgos que plantea la IA, en particular los sistemas de IA de alto riesgo, y mejorar la transparencia, responsabilidad y confianza en la IA, al tiempo que se promueve la innovación y la competitividad."

},

# Question 1 from DMA

{

    'law': 'dma',

    'question': "¿Qué criterios se utilizan para definir a un 'guardián' bajo la Ley de Mercados Digitales?",

    'answer': "Un guardián según la Ley de Mercados Digitales (DMA) se define como un proveedor de servicios de plataforma central que tiene un impacto significativo en el mercado interno, sirve como una puerta de entrada importante para que los usuarios comerciales lleguen a los usuarios finales y disfruta de una posición arraigada y duradera en el mercado. Los criterios incluyen tener una posición económica fuerte, un gran número de usuarios y control sobre un ecosistema que es difícil de contestar para otras empresas."

},

# Question 1 from DSA

{

    'law': 'dsa',

    'question': "¿Cuáles son las principales responsabilidades de las plataformas en línea bajo la Ley de Servicios Digitales?",

    'answer': "Bajo la DSA, las plataformas en línea son responsables de tomar medidas efectivas para mitigar los riesgos relacionados con el contenido ilegal, garantizar la seguridad de los usuarios y proteger los derechos fundamentales. Las plataformas deben implementar mecanismos para informar y eliminar contenido ilegal, proporcionar a los usuarios términos y condiciones claros, y establecer procesos para manejar quejas y apelaciones. Las plataformas que alcanzan un número significativo de usuarios también están obligadas a evaluar y mitigar riesgos sistémicos, como la difusión de desinformación y contenido dañino."

},

# Question 2 from GDPR

{

    'law': 'gdpr',

    'question': "¿Cómo pretende el RGPD equilibrar el derecho a la protección de datos personales con otros derechos fundamentales?",

    'answer': "Este Reglamento respeta todos los derechos fundamentales y observa las libertades y principios reconocidos en la Carta, tal como están consagrados en los Tratados, en particular el respeto a la vida privada y familiar, el domicilio y las comunicaciones, la protección de los datos personales, la libertad de pensamiento, conciencia y religión, la libertad de expresión e información, la libertad de empresa, el derecho a un recurso efectivo y a un juicio justo, y la diversidad cultural, religiosa y lingüística. El derecho a la protección de los datos personales debe considerarse en relación con su función en la sociedad y equilibrarse con otros derechos fundamentales, de conformidad con el principio de proporcionalidad."

},

# Question 2 from AI Act

{

    'law': 'ai_act',

    'question': "¿Cómo propone la Ley de IA regular los sistemas de IA de alto riesgo?",

    'answer': "La Ley de IA clasifica los sistemas de IA en función del riesgo que representan y somete a los sistemas de IA de alto riesgo a requisitos estrictos. Los sistemas de IA de alto riesgo incluyen aquellos utilizados en infraestructura crítica, educación, empleo, servicios públicos y privados esenciales, aplicación de la ley y gestión de migración, asilo y control de fronteras. Estos sistemas deben cumplir con requisitos relacionados con la gestión de riesgos, la gobernanza de datos, la documentación técnica, el mantenimiento de registros, la transparencia, la provisión de información a los usuarios, la supervisión humana, la precisión y la robustez. Los proveedores de estos sistemas deben establecer un sistema de gestión de calidad y garantizar la vigilancia continua y la supervisión posterior al mercado."

},

# Question 2 from DMA

{

    'law': 'dma',

    'question': "¿Cómo propone la DMA regular el comportamiento de los guardianes en los mercados digitales?",

    'answer': "La DMA impone obligaciones específicas a los guardianes para evitar que se involucren en prácticas desleales que perjudiquen la competencia y a los consumidores. Esto incluye prohibir a los guardianes favorecer sus propios servicios sobre los de los competidores (autopreferencia), exigirles que permitan la interoperabilidad con servicios de terceros, y garantizar que no limiten injustamente el acceso a sus plataformas. Los guardianes también están obligados a proporcionar portabilidad de datos, ofrecer términos justos a los usuarios comerciales y garantizar la transparencia en sus operaciones."

},

# Question 2 from DSA

{

    'law': 'dsa',

    'question': "¿Cómo pretende la DSA proteger a los usuarios del contenido ilegal en las plataformas digitales?",

    'answer': "La DSA pretende proteger a los usuarios del contenido ilegal al requerir que las plataformas implementen mecanismos de notificación y acción, permitiendo que los usuarios informen fácilmente sobre contenido ilegal. Las plataformas deben actuar rápidamente para eliminar o deshabilitar el acceso al contenido ilegal tras recibir una notificación. La DSA también introduce obligaciones para que las plataformas cooperen con las autoridades y proporcionen informes de transparencia sobre sus actividades de moderación de contenido. Las plataformas deben tomar medidas proactivas para prevenir la difusión de contenido ilegal y garantizar que sus algoritmos no promuevan contenido dañino o ilegal."

},

# Question 3 from GDPR

{

    'law': 'gdpr',

    'question': "¿Qué desafíos han surgido debido a los desarrollos tecnológicos y la globalización en el contexto de la protección de datos personales?",

    'answer': "Los desarrollos tecnológicos y la globalización han traído nuevos desafíos para la protección de los datos personales. La escala de la recopilación y el intercambio de datos personales ha aumentado significativamente. La tecnología permite a las empresas privadas y a las autoridades públicas utilizar datos personales a una escala sin precedentes para llevar a cabo sus actividades. Las personas físicas cada vez más ponen a disposición pública y global información personal. La tecnología ha transformado tanto la economía como la vida social, y debe facilitar aún más el libre flujo de datos personales dentro de la Unión y la transferencia a terceros países y organizaciones internacionales, garantizando al mismo tiempo un alto nivel de protección de los datos personales."

},

# Question 3 from AI Act

{

    'law': 'ai_act',

    'question': "¿Qué responsabilidades impone la Ley de IA a los proveedores de IA para garantizar prácticas éticas de IA?",

    'answer': "Los proveedores de sistemas de IA de alto riesgo son responsables de garantizar que sus sistemas cumplan con los requisitos establecidos en la Ley. Esto incluye la obligación de realizar una evaluación de conformidad antes de comercializar el sistema, asegurar que el sistema pase por pruebas adecuadas, proporcionar instrucciones e información claras a los usuarios, implementar medidas de supervisión humana y monitorear el sistema a lo largo de su ciclo de vida. Los proveedores también deben informar de incidentes graves y fallos a las autoridades."

},

# Question 3 from DMA

{

    'law': 'dma',

    'question': "¿Cuáles son las obligaciones clave impuestas a los guardianes por la DMA?",

    'answer': "Las obligaciones clave para los guardianes bajo la DMA incluyen prohibiciones sobre la combinación de datos personales de diferentes fuentes sin el consentimiento del usuario, restricciones sobre la preinstalación de software o aplicaciones, y requisitos para permitir que los usuarios comerciales accedan a los datos generados en su plataforma. Los guardianes deben asegurar que sus plataformas sean abiertas e interoperables con servicios de terceros y tienen prohibido usar datos no públicos de sus usuarios comerciales para competir contra ellos."

},

# Question 3 from DSA

{

    'law': 'dsa',

    'question': "¿Qué requisitos de transparencia se imponen a las plataformas en línea por la DSA?",

    'answer': "La DSA impone requisitos extensos de transparencia a las plataformas en línea, incluyendo la obligación de publicar informes de transparencia que detallen el número de acciones de eliminación de contenido, las razones de estas acciones y los resultados de las apelaciones de los usuarios. Las plataformas también deben divulgar cómo funcionan sus sistemas de moderación de contenido y algoritmos de recomendación, incluyendo los criterios utilizados para clasificar y mostrar contenido. Los usuarios deben ser informados sobre los términos y condiciones que rigen el uso de la plataforma y cualquier cambio realizado en estos términos. Además, las plataformas deben proporcionar información clara sobre la publicidad que sirven, incluyendo la identidad de los anunciantes y los criterios de segmentación utilizados."

},

# Question 4 from GDPR

{

    'law': 'gdpr',

    'question': "¿Cómo aborda el RGPD la transferencia de datos personales a terceros países u organizaciones internacionales?",

    'answer': "La transferencia de datos personales a terceros países u organizaciones internacionales solo se permite cuando se cumplen las condiciones establecidas en este Reglamento, con el fin de garantizar que el nivel de protección de las personas físicas garantizado por este Reglamento no se vea menoscabado. En cualquier caso, las transferencias a terceros países y organizaciones internacionales solo pueden llevarse a cabo en pleno cumplimiento de este Reglamento. Este Reglamento se entiende sin perjuicio de los acuerdos internacionales celebrados entre la Unión y terceros países que regulen la transferencia de datos personales, incluidas las garantías adecuadas para los interesados."

},

# Question 4 from AI Act

{

    'law': 'ai_act',

    'question': "¿Cómo aborda la Ley de IA la transparencia y la responsabilidad en los sistemas de IA?",

    'answer': "La Ley de IA exige que los sistemas de IA, especialmente los de alto riesgo, sean transparentes y proporcionen información clara sobre su propósito, capacidades y limitaciones. Los usuarios deben poder comprender cómo se toman las decisiones por los sistemas de IA y qué datos se están procesando. La Ley exige que los sistemas de IA se diseñen con características que aseguren la responsabilidad, incluida la auditabilidad, la trazabilidad de las decisiones y la capacidad de proporcionar explicaciones de las decisiones tomadas por la IA."

},

# Question 4 from DMA

{

    'law': 'dma',

    'question': "¿Cómo pretende la DMA prevenir prácticas desleales en el mercado digital?",

    'answer': "La DMA pretende prevenir prácticas desleales estableciendo reglas claras para los guardianes, incluyendo prohibiciones sobre la autopreferencia, restricciones en términos y condiciones injustas para los usuarios comerciales, y requisitos de transparencia en cómo operan. La DMA también garantiza que los guardianes no puedan usar su posición dominante para sofocar la competencia o la innovación por parte de empresas más pequeñas. La Comisión Europea tiene el poder de investigar y sancionar a los guardianes que no cumplan con estas reglas."

},

# Question 4 from DSA

{

    'law': 'dsa',

    'question': "¿Cómo propone la DSA manejar la difusión de contenido dañino?",

    'answer': "La DSA propone manejar la difusión de contenido dañino al requerir que las plataformas evalúen los riesgos asociados con la difusión de contenido dañino o ilegal y tomen medidas apropiadas para mitigar estos riesgos. Las plataformas deben implementar salvaguardas para asegurar que sus algoritmos no promuevan contenido dañino, y deben proporcionar a los usuarios herramientas para controlar el contenido al que están expuestos. La DSA también alienta a las plataformas a cooperar con denunciantes de confianza y verificadores de hechos para identificar y abordar más eficazmente el contenido dañino. En casos donde las plataformas no mitiguen adecuadamente los riesgos, pueden estar sujetas a acciones regulatorias, incluidas multas y otras sanciones."

},

# Question 5 from GDPR

{

    'law': 'gdpr',

    'question': "¿Qué protecciones específicas ofrece el RGPD a los niños con respecto a sus datos personales?",

    'answer': "Los niños merecen una protección específica con respecto a sus datos personales, ya que pueden ser menos conscientes de los riesgos, las consecuencias, las salvaguardias y los derechos en relación con el tratamiento de datos personales. Esta protección específica debe aplicarse, en particular, al uso de datos personales de niños con fines de marketing o para crear perfiles de personalidad o de usuario, así como a la recopilación de datos personales con respecto a los niños cuando utilizan servicios ofrecidos directamente a un niño. El consentimiento del titular de la responsabilidad parental no debe ser necesario en el contexto de servicios preventivos o de asesoramiento ofrecidos directamente a un niño."

},

# Question 5 from AI Act

{

    'law': 'ai_act',

    'question': "¿Qué medidas sugiere la Ley de IA para proteger los derechos fundamentales en el despliegue de tecnologías de IA?",

    'answer': "La Ley de IA incorpora varias medidas para proteger los derechos fundamentales, como exigir que los sistemas de IA se diseñen y utilicen de manera coherente con el respeto a la dignidad humana, la privacidad, la no discriminación y otros derechos fundamentales. Esto incluye incorporar mecanismos de supervisión humana, asegurarse de que los sistemas de IA no conduzcan a resultados sesgados o discriminatorios y proporcionar vías para que las personas impugnen las decisiones tomadas por sistemas de IA que los afecten significativamente. La Ley también promueve el desarrollo de códigos de conducta y medidas voluntarias por parte de los proveedores para asegurar que la IA se use de manera ética y en alineación con los valores sociales."

},

# Question 5 from DMA

{

    'law': 'dma',

    'question': "¿Qué mecanismos de aplicación están incluidos en la DMA para garantizar el cumplimiento por parte de los guardianes?",

    'answer': "La DMA incluye mecanismos de aplicación robustos, como la capacidad de la Comisión Europea para imponer multas de hasta el 10% del volumen de negocios total mundial anual del guardián por incumplimiento. En casos de infracciones repetidas, la Comisión puede imponer sanciones adicionales, incluidos remedios estructurales, como la desinversión de negocios. La DMA también permite pagos de multas periódicas para asegurar que los guardianes cumplan con las obligaciones y prohibiciones establecidas en la regulación."

},

# Question 5 from DSA

{

    'law': 'dsa',

    'question': "¿Qué medidas incluye la DSA para proteger la libertad de expresión mientras combate el contenido ilegal?",

    'answer': "La DSA incluye medidas para proteger la libertad de expresión al asegurar que cualquier restricción sobre el contenido sea necesaria, proporcionada y legalmente justificada. Las plataformas deben proporcionar a los usuarios explicaciones claras cuando el contenido sea eliminado o el acceso sea restringido, y los usuarios deben tener el derecho de apelar tales decisiones. La DSA también requiere que las plataformas aseguren que los procesos de moderación de contenido sean justos y transparentes, con salvaguardias en su lugar para prevenir la eliminación arbitraria de contenido. Además, la DSA alienta a las plataformas a desarrollar códigos de conducta en colaboración con las partes interesadas para equilibrar la necesidad de combatir el contenido ilegal con la protección de la libertad de expresión."

},

# Question 6 from GDPR

{

    'law': 'gdpr',

    'question': "¿Cómo define el RGPD los datos personales y cuáles son algunos ejemplos?",

    'answer': "Los datos personales según el RGPD se definen como cualquier información relacionada con una persona física identificada o identificable (‘sujeto de datos’). Ejemplos incluyen el nombre de una persona, número de identificación, datos de localización, un identificador en línea o uno o más factores específicos de la identidad física, fisiológica, genética, mental, económica, cultural o social de esa persona física. La definición es amplia, abarcando diversas formas de datos que podrían usarse para identificar directa o indirectamente a un individuo."

},

# Question 6 from AI Act

{

    'law': 'ai_act',

    'question': "¿Qué categorías de sistemas de IA se consideran de alto riesgo según la Ley de IA?",

    'answer': "Los sistemas de IA de alto riesgo según la Ley de IA incluyen aquellos utilizados en infraestructura crítica (como transporte, energía y suministro de agua), educación y formación profesional, empleo y gestión de trabajadores, acceso a servicios privados y públicos esenciales (como la calificación crediticia y los beneficios sociales), aplicación de la ley (como la vigilancia predictiva), gestión de migración, asilo y control de fronteras, y administración de justicia y procesos democráticos. Estos sistemas están sujetos a requisitos estrictos debido a los riesgos significativos que representan para los derechos fundamentales y la seguridad."

},

# Question 6 from DMA

{

    'law': 'dma',

    'question': "¿Cómo aborda la DMA la cuestión de la autopreferencia por parte de los guardianes?",

    'answer': "La DMA prohíbe específicamente a los guardianes participar en prácticas de autopreferencia, donde favorecen sus propios productos o servicios sobre los de los competidores en sus plataformas. Esto incluye prácticas como clasificar sus propios productos más alto en los resultados de búsqueda o dar acceso preferente a datos. El objetivo es asegurar un campo de juego nivelado en los mercados digitales, donde la competencia se base en el mérito en lugar del poder de mercado del guardián. La prohibición de la autopreferencia es una de las obligaciones clave impuestas a los guardianes para prevenir el comportamiento anticompetitivo."

},

# Question 6 from DSA

{

    'law': 'dsa',

    'question': "¿Cómo aborda la DSA la cuestión de la moderación de contenido en las plataformas en línea?",

    'answer': "La DSA requiere que las plataformas en línea implementen políticas de moderación de contenido que sean transparentes, consistentes y alineadas con los derechos fundamentales. Las plataformas deben establecer términos y condiciones claros para la moderación de contenido y proporcionar a los usuarios información detallada sobre cómo se evalúa, elimina o restringe el contenido. La DSA también obliga a las plataformas a implementar mecanismos para que los usuarios apelen las decisiones de moderación de contenido, asegurando que los usuarios tengan la oportunidad de impugnar eliminaciones o restricciones injustificadas. Estas medidas están destinadas a crear un sistema de moderación de contenido justo y responsable que respete la libertad de expresión mientras combate el contenido ilegal."

},

# Question 7 from GDPR

{

    'law': 'gdpr',

    'question': "¿Cuál es la base legal para el tratamiento de datos personales bajo el RGPD?",

    'answer': "El RGPD establece varias bases legales para el tratamiento de datos personales, incluyendo: el consentimiento del interesado para el tratamiento; el tratamiento es necesario para la ejecución de un contrato en el que el interesado es parte; el tratamiento es necesario para el cumplimiento de una obligación legal; el tratamiento es necesario para proteger los intereses vitales del interesado o de otra persona física; el tratamiento es necesario para el desempeño de una tarea realizada en interés público o en el ejercicio de la autoridad oficial; y el tratamiento es necesario para los intereses legítimos perseguidos por el responsable del tratamiento o un tercero, salvo que dichos intereses queden anulados por los intereses o derechos y libertades fundamentales del interesado."

},

# Question 7 from AI Act

{

    'law': 'ai_act',

    'question': "¿Cómo define la Ley de IA 'sistema de IA' y qué tecnologías entran bajo esta definición?",

    'answer': "La Ley de IA define un 'sistema de IA' como software que se desarrolla con una o más de las técnicas y enfoques enumerados en la Ley, como el aprendizaje automático, los enfoques basados en la lógica y el conocimiento, y los enfoques estadísticos. Estos sistemas pueden, para un conjunto dado de objetivos definidos por humanos, generar resultados como contenido, predicciones, recomendaciones o decisiones que influyen en los entornos con los que interactúan. La definición es amplia e incluye una variedad de tecnologías de IA, desde algoritmos simples hasta modelos complejos de aprendizaje automático."

},

# Question 7 from DMA

{

    'law': 'dma',

    'question': "¿Cuáles son los criterios para identificar servicios de plataforma central bajo la DMA?",

    'answer': "Los servicios de plataforma central bajo la DMA incluyen una gama de servicios digitales que sirven como puertas de entrada importantes para que los usuarios comerciales lleguen a los usuarios finales. Estos servicios incluyen servicios de intermediación en línea, como tiendas de aplicaciones y mercados, motores de búsqueda en línea, servicios de redes sociales, servicios de plataformas de intercambio de videos, servicios de comunicación interpersonal independientes del número, sistemas operativos, servicios de computación en la nube y servicios de publicidad. Un servicio se considera un servicio de plataforma central si tiene un impacto significativo en el mercado interno y es una puerta de entrada esencial para que los usuarios comerciales accedan a los usuarios finales."

},

# Question 7 from DSA

{

    'law': 'dsa',

    'question': "¿Qué obligaciones tienen las plataformas en línea muy grandes (VLOPs) bajo la DSA?",

    'answer': "Las plataformas en línea muy grandes (VLOPs), definidas como plataformas con más de 45 millones de usuarios en la UE, tienen obligaciones adicionales bajo la DSA debido a su impacto significativo en la sociedad y el discurso público. Las VLOPs deben realizar evaluaciones de riesgos anuales para identificar y mitigar riesgos sistémicos, como la difusión de contenido ilegal, desinformación y contenido dañino. También están obligadas a proporcionar una mayor transparencia en sus algoritmos de recomendación de contenido, ofrecer a los usuarios más control sobre el contenido que ven y cooperar con las autoridades para prevenir y abordar los riesgos sistémicos. Estas obligaciones están destinadas a asegurar que las VLOPs operen de manera segura, transparente y respetuosa con los derechos fundamentales."

},

# Question 8 from GDPR

{

    'law': 'gdpr',

    'question': "¿Cuáles son los derechos de los interesados bajo el RGPD?",

    'answer': "El RGPD otorga a los interesados varios derechos, incluidos el derecho a ser informado, el derecho de acceso, el derecho de rectificación, el derecho de supresión (‘derecho al olvido’), el derecho a la limitación del tratamiento, el derecho a la portabilidad de los datos, el derecho a oponerse al tratamiento y derechos relacionados con la toma de decisiones automatizada y la elaboración de perfiles. Estos derechos permiten a las personas tener control sobre sus datos personales y garantizan la transparencia y responsabilidad en el tratamiento de datos."

},

# Question 8 from AI Act

{

    'law': 'ai_act',

    'question': "¿Qué obligaciones tienen los usuarios de sistemas de IA de alto riesgo bajo la Ley de IA?",

    'answer': "Los usuarios de sistemas de IA de alto riesgo están obligados a operar los sistemas de acuerdo con las instrucciones proporcionadas por el proveedor del sistema de IA, monitorear el funcionamiento del sistema de IA e informar de inmediato cualquier incidente grave o fallo al proveedor y a las autoridades competentes. Los usuarios también deben mantener los registros generados por el sistema de IA, garantizar que se mantenga la supervisión humana y asegurarse de que el sistema de IA se utilice solo para su propósito previsto. Además, los usuarios son responsables de implementar medidas para mitigar los riesgos para los derechos fundamentales y la seguridad."

},

# Question 8 from DMA

{

    'law': 'dma',

    'question': "¿Cómo promueve la DMA la interoperabilidad entre servicios digitales?",

    'answer': "La DMA promueve la interoperabilidad al exigir a los guardianes que aseguren que sus servicios de plataforma central puedan interactuar con servicios de terceros. Esto incluye poner a disposición las interfaces técnicas y la documentación necesarias para permitir la interoperabilidad. El objetivo es evitar que los guardianes bloqueen a los usuarios y usuarios comerciales en sus plataformas y permitir la competencia al permitir que nuevos entrantes y competidores más pequeños ofrezcan servicios complementarios o competitivos. La interoperabilidad se considera una medida clave para promover la innovación y la elección del consumidor en los mercados digitales."

},

# Question 8 from DSA

{

    'law': 'dsa',

    'question': "¿Cómo mejora la DSA la protección de los menores en línea?",

    'answer': "La DSA incluye disposiciones específicas para mejorar la protección de los menores en línea, reconociendo que los niños son particularmente vulnerables al contenido y prácticas dañinas. Las plataformas deben implementar medidas para asegurar que sus servicios sean seguros para los menores, incluyendo la moderación de contenido adecuada para la edad, controles parentales y restricciones sobre la publicidad dirigida a menores. La DSA también requiere que las plataformas proporcionen información clara y accesible a los menores y a sus padres sobre los riesgos asociados con las actividades en línea y cómo protegerse. Estas medidas están diseñadas para crear un entorno en línea más seguro para los niños y para empoderarlos a ellos y a sus tutores para que tomen decisiones informadas."

},

# Question 9 from GDPR

{

    'law': 'gdpr',

    'question': "¿Cómo aborda el RGPD la protección de datos por diseño y por defecto?",

    'answer': "El RGPD requiere que los responsables del tratamiento implementen la protección de datos por diseño y por defecto. Esto significa que las medidas de protección de datos deben integrarse en las actividades de tratamiento desde el principio y que solo se procesan los datos personales necesarios para cada propósito específico del tratamiento. El responsable del tratamiento debe adoptar medidas técnicas y organizativas adecuadas, como la seudonimización, para garantizar que, por defecto, los datos personales no se hagan accesibles a un número indefinido de personas sin el consentimiento del individuo."

},

# Question 9 from AI Act

{

    'law': 'ai_act',

    'question': "¿Cómo aborda la Ley de IA el uso de sistemas de identificación biométrica?",

    'answer': "La Ley de IA impone regulaciones estrictas sobre el uso de sistemas de identificación biométrica, especialmente aquellos utilizados en espacios públicos con fines de aplicación de la ley. El uso de sistemas de identificación biométrica remota en tiempo real en espacios accesibles al público está generalmente prohibido, con excepciones bajo condiciones específicas, como prevenir un ataque terrorista, localizar a un niño desaparecido o identificar a un sospechoso de un delito grave. Incluso en estos casos, el uso debe ser autorizado por autoridades judiciales u otras autoridades independientes y sujeto a estrictas salvaguardias para proteger los derechos fundamentales."

},

# Question 9 from DMA

{

    'law': 'dma',

    'question': "¿Qué obligaciones impone la DMA a los guardianes en relación con el acceso y la portabilidad de datos?",

    'answer': "La DMA impone obligaciones a los guardianes para proporcionar a los usuarios comerciales y a los usuarios finales acceso a los datos generados a través de sus interacciones en la plataforma. Esto incluye proporcionar datos en un formato estructurado, de uso común y legible por máquina para facilitar la portabilidad de datos. Los guardianes también están obligados a permitir que los usuarios comerciales accedan a datos que son necesarios para el desarrollo y mejora de sus propios productos y servicios. Estas obligaciones están destinadas a evitar que los guardianes utilicen su control sobre los datos para sofocar la competencia y la innovación."

},

# Question 9 from DSA

{

    'law': 'dsa',

    'question': "¿Cuáles son las obligaciones de transparencia para las plataformas en línea en cuanto a sus algoritmos?",

    'answer': "La DSA impone obligaciones de transparencia a las plataformas en línea para proporcionar información clara y accesible sobre cómo funcionan sus algoritmos, particularmente aquellos utilizados para la moderación, recomendación y clasificación de contenido. Las plataformas deben explicar los criterios y la lógica detrás de sus algoritmos, permitiendo a los usuarios entender cómo se toman las decisiones y cómo se les presenta el contenido. Las VLOPs tienen obligaciones adicionales para realizar auditorías algorítmicas y permitir que investigadores independientes evalúen el impacto de sus algoritmos en la sociedad. Estas medidas de transparencia están destinadas a aumentar la responsabilidad y la confianza en el ecosistema digital."

},

# Question 10 from GDPR

{

    'law': 'gdpr',

    'question': "¿Cuál es el papel del Delegado de Protección de Datos (DPD) bajo el RGPD?",

    'answer': "El Delegado de Protección de Datos (DPD) es responsable de supervisar las estrategias de protección de datos y garantizar el cumplimiento de los requisitos del RGPD. El DPD debe ser designado por autoridades y organismos públicos, y por organizaciones que realicen un seguimiento regular y sistemático de los interesados a gran escala o procesen categorías especiales de datos a gran escala. Las responsabilidades del DPD incluyen asesorar a la organización sobre las obligaciones del RGPD, monitorear el cumplimiento, proporcionar capacitación al personal, realizar auditorías y servir como punto de contacto para las autoridades de control y los interesados."

},

# Question 10 from AI Act

{

    'law': 'ai_act',

    'question': "¿Cuáles son los requisitos para las evaluaciones de conformidad bajo la Ley de IA?",

    'answer': "Los sistemas de IA de alto riesgo deben someterse a una evaluación de conformidad antes de que puedan ser comercializados o puestos en servicio. Esta evaluación implica evaluar si el sistema de IA cumple con los requisitos establecidos en la Ley de IA, incluidos la gestión de riesgos, la gobernanza de datos, la transparencia, la supervisión humana y la precisión. La evaluación puede ser realizada por el proveedor o por un organismo notificado, según la naturaleza del sistema de IA. La evaluación de conformidad debe estar documentada y el sistema de IA debe llevar el marcado CE que indique el cumplimiento de la regulación."

},

# Question 10 from DMA

{

    'law': 'dma',

    'question': "¿Cómo aborda la DMA la cuestión de las prácticas de vinculación y agrupación por parte de los guardianes?",

    'answer': "La DMA prohíbe a los guardianes participar en prácticas de vinculación y agrupación que requieran a los usuarios comprar o usar servicios adicionales como condición para acceder al servicio de plataforma central del guardián. Por ejemplo, un guardián no puede requerir que los usuarios instalen o usen una aplicación o servicio específico como condición para usar su plataforma. La prohibición de la vinculación y la agrupación está destinada a evitar que los guardianes utilicen su poder de mercado para extender su dominio a otros mercados y garantizar que los usuarios tengan la libertad de elegir los servicios que desean utilizar."

},

# Question 10 from DSA

{

    'law': 'dsa',

    'question': "¿Cómo aborda la DSA la cuestión de la desinformación y las noticias falsas en las plataformas digitales?",

    'answer': "La DSA requiere que las plataformas, particularmente las VLOPs, tomen medidas proactivas para combatir la difusión de desinformación y noticias falsas. Esto incluye implementar mecanismos para detectar, evaluar y mitigar los riesgos asociados con la desinformación, colaborar con verificadores de hechos independientes y proporcionar a los usuarios información precisa y contexto. Las plataformas también deben asegurar que sus sistemas de moderación de contenido y recomendación no amplifiquen ni promuevan la desinformación. La DSA promueve la transparencia al requerir que las plataformas informen sobre sus esfuerzos para combatir la desinformación y proporcionen a los usuarios herramientas para identificar e informar sobre información falsa."

},

# Question 11 from GDPR

{

    'law': 'gdpr',

    'question': "¿Cuáles son las implicaciones del RGPD para las actividades de tratamiento de datos transfronterizas?",

    'answer': "El RGPD establece un marco para las actividades de tratamiento de datos transfronterizas para asegurar que la protección de datos sea consistente en toda la UE. Las organizaciones que procesan datos personales en varios estados miembros de la UE deben designar una autoridad de control principal, que actúe como punto único de contacto para supervisar el cumplimiento. El RGPD también facilita la cooperación entre las autoridades de control a través de mecanismos como el mecanismo de coherencia y el Comité Europeo de Protección de Datos (CEPD)."

},

# Question 11 from AI Act

{

    'law': 'ai_act',

    'question': "¿Qué papel juegan las autoridades de supervisión nacionales bajo la Ley de IA?",

    'answer': "Las autoridades de supervisión nacionales son responsables de supervisar la implementación y la aplicación de la Ley de IA dentro de sus respectivas jurisdicciones. Están encargadas de monitorear el cumplimiento de los sistemas de IA con los requisitos de la Ley, realizar inspecciones e investigaciones y tomar medidas de ejecución cuando sea necesario. Estas autoridades también desempeñan un papel clave en la coordinación con otras autoridades nacionales y la Comisión Europea para garantizar un enfoque armonizado de la regulación de la IA en toda la UE."

},

# Question 11 from DMA

{

    'law': 'dma',

    'question': "¿Cuáles son las consecuencias para los guardianes que no cumplan con la DMA?",

    'answer': "Los guardianes que no cumplan con las obligaciones y prohibiciones establecidas en la DMA enfrentan consecuencias significativas, incluidas multas de hasta el 10% de su volumen de negocios total mundial anual. En casos de incumplimiento repetido, la Comisión Europea puede imponer medidas adicionales, como remedios estructurales, incluida la desinversión de partes del negocio. La DMA también prevé pagos de multas periódicas para asegurar que los guardianes cumplan con las obligaciones de manera continua. La aplicación de la DMA está diseñada para ser robusta para prevenir que los guardianes participen en comportamientos anticompetitivos."

},

# Question 11 from DSA

{

    'law': 'dsa',

    'question': "¿Qué papel juegan los denunciantes de confianza bajo la DSA?",

    'answer': "La DSA reconoce el papel de los denunciantes de confianza, entidades con experiencia en la identificación de contenido ilegal, como socios importantes en la moderación de contenido. A los denunciantes de confianza se les concede prioridad en los mecanismos de notificación y acción, lo que significa que sus informes se procesan más rápidamente y con mayor precisión. Las plataformas deben asegurar que los informes de los denunciantes de confianza sean manejados por moderadores experimentados y que reciban retroalimentación sobre las acciones tomadas. La designación de denunciantes de confianza tiene como objetivo mejorar la eficiencia y efectividad de la moderación de contenido, particularmente en la lucha contra el contenido ilegal y actividades dañinas en línea."

},

# Question 12 from GDPR

{

    'law': 'gdpr',

    'question': "¿Cómo maneja el RGPD las violaciones de datos y cuáles son las obligaciones de los responsables del tratamiento en tales casos?",

    'answer': "Bajo el RGPD, los responsables del tratamiento deben notificar las violaciones de datos a la autoridad de control competente en un plazo de 72 horas desde que tengan conocimiento de la violación, a menos que sea improbable que la violación resulte en un riesgo para los derechos y libertades de las personas. Si la violación supone un alto riesgo para los individuos afectados, el responsable del tratamiento también debe informar a los interesados sin demora injustificada. El RGPD exige que las organizaciones implementen medidas técnicas y organizativas adecuadas para prevenir las violaciones de datos y mitigar su impacto."

},

# Question 12 from AI Act

{

    'law': 'ai_act',

    'question': "¿Cómo fomenta la Ley de IA la innovación asegurando al mismo tiempo la seguridad y el cumplimiento?",

    'answer': "La Ley de IA fomenta la innovación proporcionando entornos regulatorios de prueba (regulatory sandboxes), que son entornos controlados donde los desarrolladores de IA pueden probar sus sistemas bajo la supervisión de autoridades competentes sin enfrentar de inmediato todos los requisitos regulatorios. Estos entornos permiten la experimentación y el desarrollo de soluciones innovadoras de IA, mientras se asegura que se mantengan los estándares de seguridad, ética y legales. La Ley también promueve la adopción de códigos de conducta voluntarios para sistemas de IA que no son de alto riesgo, permitiendo que los proveedores demuestren su compromiso con prácticas éticas de IA."

},

# Question 12 from DMA

{

    'law': 'dma',

    'question': "¿Cómo mejora la DMA la protección del consumidor en los mercados digitales?",

    'answer': "La DMA mejora la protección del consumidor al garantizar que los guardianes no participen en prácticas que perjudiquen a los consumidores, como la autopreferencia, los términos y condiciones injustos o la limitación del acceso a los datos. La DMA también promueve la transparencia en cómo operan los guardianes, exigiéndoles que proporcionen información clara y accesible a los consumidores sobre sus prácticas. Además, la DMA asegura que los consumidores tengan más opciones y control sobre los servicios digitales que utilizan, promoviendo la interoperabilidad y la portabilidad de datos. Al fomentar la competencia, la DMA busca mejorar la calidad y la asequibilidad de los servicios digitales para los consumidores."

},

# Question 12 from DSA

{

    'law': 'dsa',

    'question': "¿Cómo promueve la DSA la responsabilidad de las plataformas en línea?",

    'answer': "La DSA promueve la responsabilidad al imponer rigurosos requisitos de informe y transparencia a las plataformas en línea. Las plataformas deben publicar informes de transparencia regulares que detallen sus actividades de moderación de contenido, incluyendo el número de acciones de eliminación, las razones de las eliminaciones y los resultados de las apelaciones de los usuarios. Las VLOPs también están obligadas a someterse a auditorías independientes de sus prácticas de moderación de contenido y gestión de riesgos. Estas auditorías están destinadas a evaluar el cumplimiento de la plataforma con la DSA e identificar áreas de mejora. Al promover la transparencia y la responsabilidad, la DSA busca generar confianza en el entorno digital y asegurar que las plataformas actúen de manera responsable."

},

# Question 13 from GDPR

{

    'law': 'gdpr',

    'question': "¿Cuáles son las restricciones para el tratamiento de categorías especiales de datos personales bajo el RGPD?",

    'answer': "El RGPD impone reglas más estrictas para el tratamiento de categorías especiales de datos personales, como datos que revelen el origen racial o étnico, opiniones políticas, creencias religiosas o filosóficas, afiliación sindical, datos genéticos, datos biométricos, datos de salud y datos relativos a la vida sexual o la orientación sexual de una persona. El tratamiento de dichos datos está prohibido a menos que se cumplan condiciones específicas, como obtener el consentimiento explícito del interesado, cumplir con obligaciones legales en el ámbito del empleo y la seguridad social, o proteger los intereses vitales del interesado."

},

# Question 13 from AI Act

{

    'law': 'ai_act',

    'question': "¿Cómo aborda la Ley de IA la transparencia de los sistemas de IA?",

    'answer': "La Ley de IA exige que los sistemas de IA, especialmente los de alto riesgo, sean diseñados y desarrollados con transparencia en mente. Esto incluye proporcionar información clara y accesible a los usuarios sobre el propósito, las capacidades, las limitaciones y el funcionamiento del sistema de IA. Los usuarios deben ser informados cuando estén interactuando con un sistema de IA, especialmente en los casos en que la IA se utiliza para tomar decisiones con impactos significativos en las personas. Los requisitos de transparencia están dirigidos a asegurar que los usuarios y las personas afectadas entiendan cómo y por qué los sistemas de IA toman decisiones."

},

# Question 13 from DMA

{

    'law': 'dma',

    'question': "¿Cómo aborda la DMA la cuestión del acceso a los datos de los usuarios comerciales por parte de los guardianes?",

    'answer': "La DMA impone obligaciones a los guardianes para proporcionar a los usuarios comerciales acceso a los datos que generan a través de sus interacciones en la plataforma. Esto incluye el acceso a datos agregados y anonimizados, así como a datos que son esenciales para el desarrollo y la mejora de los productos y servicios del usuario comercial. La DMA también prohíbe a los guardianes usar datos no públicos de los usuarios comerciales para competir contra ellos, asegurando que los guardianes no exploten su acceso a los datos para obtener una ventaja competitiva injusta."

},

# Question 13 from DSA

{

    'law': 'dsa',

    'question': "¿Cuáles son las sanciones por incumplimiento de la DSA?",

    'answer': "La DSA prevé sanciones sustanciales por incumplimiento, incluidas multas de hasta el 6% del volumen de negocios total anual mundial de la plataforma. En casos de incumplimiento repetido o grave, la DSA permite medidas adicionales, como la suspensión temporal de los servicios de la plataforma u otras acciones correctivas. La aplicación de la DSA está supervisada por autoridades reguladoras nacionales, que tienen el poder de investigar y sancionar a las plataformas que violen la regulación. Estas sanciones están diseñadas para asegurar que las plataformas tomen en serio sus obligaciones y que las disposiciones de la DSA se implementen efectivamente."

},

# Question 14 from GDPR

{

    'law': 'gdpr',

    'question': "¿Cómo regula el RGPD la toma de decisiones automatizadas y la elaboración de perfiles?",

    'answer': "El RGPD impone restricciones a la toma de decisiones automatizada, incluida la elaboración de perfiles, cuando las decisiones se toman únicamente en base al tratamiento automatizado y afectan significativamente a las personas. Dicho tratamiento solo se permite en situaciones específicas, como cuando es necesario para la ejecución de un contrato, autorizado por la ley de la Unión o del Estado miembro, o basado en el consentimiento explícito del interesado. Las organizaciones deben asegurarse de que las personas sean informadas sobre la existencia de decisiones automatizadas, la lógica involucrada y las posibles consecuencias. Los interesados tienen derecho a impugnar las decisiones automatizadas y solicitar intervención humana."

},

# Question 14 from AI Act

{

    'law': 'ai_act',

    'question': "¿Cuáles son las obligaciones relacionadas con la calidad de los datos bajo la Ley de IA?",

    'answer': "La Ley de IA requiere que los sistemas de IA de alto riesgo sean entrenados, probados y validados utilizando conjuntos de datos de alta calidad que sean relevantes, representativos, libres de errores y completos. Los datos deben ser seleccionados cuidadosamente para evitar sesgos que podrían llevar a resultados discriminatorios. Los proveedores deben asegurarse de que el marco de gobernanza de datos incluya medidas para evaluar y mitigar los riesgos relacionados con la calidad de los datos, como el uso de conjuntos de datos diversos y representativos, validar la precisión y fiabilidad de los datos, y actualizar regularmente los conjuntos de datos para reflejar los cambios a lo largo del tiempo."

},

# Question 14 from DMA

{

    'law': 'dma',

    'question': "¿Cómo asegura la DMA el acceso justo y no discriminatorio a los servicios de plataforma central?",

    'answer': "La DMA requiere que los guardianes aseguren que sus servicios de plataforma central se ofrezcan en términos justos, razonables y no discriminatorios. Esto significa que los guardianes no pueden imponer términos o condiciones injustas a los usuarios comerciales ni participar en prácticas que favorezcan sus propios servicios sobre los de los competidores. La DMA también requiere que los guardianes proporcionen transparencia en cómo operan, incluida información clara y accesible sobre los términos y condiciones para usar sus servicios. Estas medidas están destinadas a prevenir que los guardianes abusen de su poder de mercado y asegurar un campo de juego nivelado en los mercados digitales."

},

# Question 14 from DSA

{

    'law': 'dsa',

    'question': "¿Cómo aborda la DSA la cuestión de los bienes, servicios y contenidos ilegales en línea?",

    'answer': "La DSA requiere que las plataformas implementen medidas para detectar y eliminar bienes, servicios y contenidos ilegales de sus servicios. Esto incluye asegurar que los vendedores y proveedores de servicios en sus plataformas estén debidamente identificados y que cumplan con las leyes y regulaciones aplicables. Las plataformas también deben proporcionar a los usuarios mecanismos claros para informar sobre bienes y servicios ilegales, y deben actuar rápidamente para eliminar o deshabilitar el acceso a dicho contenido. Las disposiciones de la DSA están diseñadas para proteger a los consumidores y asegurar que los mercados en línea operen de manera segura y legal."

},

# Question 15 from GDPR

{

    'law': 'gdpr',

    'question': "¿Qué sanciones y medidas de ejecución se prevén en el RGPD?",

    'answer': "El RGPD prevé sanciones sustanciales y medidas de ejecución para garantizar el cumplimiento. Las autoridades de control tienen el poder de imponer multas administrativas de hasta 20 millones de euros o el 4% del volumen de negocios anual total mundial del ejercicio financiero anterior, lo que sea mayor, por las violaciones más graves. Las sanciones se determinan en función de factores como la naturaleza, gravedad y duración de la infracción, el carácter intencional o negligente de la infracción, y las medidas adoptadas por la organización para mitigar el daño."

},

# Question 15 from AI Act

{

    'law': 'ai_act',

    'question': "¿Cómo regula la Ley de IA el uso de la IA en la aplicación de la ley y la seguridad pública?",

    'answer': "La Ley de IA impone estrictas regulaciones sobre el uso de sistemas de IA en la aplicación de la ley y la seguridad pública, especialmente aquellos utilizados para la vigilancia predictiva, la identificación biométrica y la vigilancia. Estos sistemas se consideran de alto riesgo y están sujetos a un riguroso escrutinio para asegurar que no infrinjan los derechos fundamentales, como la privacidad y la no discriminación. Las agencias de aplicación de la ley deben realizar una evaluación de riesgos detallada e implementar salvaguardias para garantizar que el uso de sistemas de IA sea necesario, proporcionado y respetuoso de los derechos humanos."

},

# Question 15 from DMA

{

    'law': 'dma',

    'question': "¿Cómo promueve la DMA la innovación y la competencia en los mercados digitales?",

    'answer': "La DMA promueve la innovación y la competencia al prevenir que los guardianes participen en prácticas que sofocan la competencia, como la autopreferencia, la vinculación y la agrupación. Al asegurar que los guardianes operen en términos justos, razonables y no discriminatorios, la DMA crea oportunidades para que nuevos entrantes y competidores más pequeños compitan en igualdad de condiciones. La DMA también promueve la interoperabilidad y la portabilidad de datos, permitiendo a las empresas desarrollar servicios innovadores que puedan interactuar con la plataforma del guardián. Estas medidas están diseñadas para fomentar un mercado digital dinámico y competitivo que beneficie a los consumidores y a las empresas por igual."

},

# Question 15 from DSA

{

    'law': 'dsa',

    'question': "¿Cómo apoya la DSA los derechos de los consumidores en el mercado digital?",

    'answer': "La DSA fortalece los derechos de los consumidores al asegurar que las plataformas en línea proporcionen información clara y accesible sobre los bienes, servicios y contenidos disponibles en sus plataformas. Esto incluye requerir que las plataformas divulguen información sobre la identidad de los vendedores, los términos y condiciones de las transacciones, y la naturaleza de los bienes y servicios ofrecidos. Los consumidores también deben ser informados sobre sus derechos, incluyendo el derecho a retirarse de una transacción, el derecho a un reembolso y el derecho a acceder a mecanismos efectivos de resolución de disputas. Las disposiciones de protección al consumidor de la DSA están diseñadas para crear un mercado digital seguro y transparente."

},

# Question 16 from GDPR

{

    'law': 'gdpr',

    'question': "¿Cuál es el papel del Comité Europeo de Protección de Datos (CEPD) bajo el RGPD?",

    'answer': "El Comité Europeo de Protección de Datos (CEPD) es un organismo independiente establecido por el RGPD para garantizar la aplicación coherente de las normas de protección de datos en toda la UE. El CEPD está compuesto por representantes de las autoridades nacionales de protección de datos y del Supervisor Europeo de Protección de Datos (SEPD). Sus responsabilidades incluyen emitir directrices, recomendaciones y buenas prácticas sobre la interpretación y aplicación del RGPD, resolver disputas entre autoridades de control y asesorar a la Comisión Europea sobre cuestiones de protección de datos."

},

# Question 16 from AI Act

{

    'law': 'ai_act',

    'question': "¿Cómo aborda la Ley de IA el problema del sesgo y la discriminación en los sistemas de IA?",

    'answer': "La Ley de IA exige que los sistemas de IA, especialmente los de alto riesgo, sean diseñados y desarrollados de manera que prevenga, identifique y mitigue los sesgos que podrían llevar a resultados discriminatorios. Los proveedores deben tomar medidas para asegurar que los sistemas de IA no produzcan resultados que desventajen injustamente a individuos o grupos basados en características protegidas como la raza, el género o la religión. Esto incluye el uso de conjuntos de datos diversos, la realización de auditorías de sesgo y la implementación de medidas correctivas para abordar los sesgos identificados. La Ley también enfatiza la importancia de la supervisión humana en la prevención y el abordaje del sesgo."

},

# Question 16 from DMA

{

    'law': 'dma',

    'question': "¿Cómo aborda la DMA la cuestión de las fusiones y adquisiciones por parte de los guardianes?",

    'answer': "La DMA requiere que los guardianes informen a la Comisión Europea sobre cualquier fusión, adquisición o concentración prevista que involucre a otros proveedores de servicios de plataforma central o servicios digitales. Este requisito de notificación permite a la Comisión evaluar si la transacción propuesta socavaría los objetivos de la DMA, como reforzar el poder de mercado del guardián o reducir la competencia en los mercados digitales. Las disposiciones de la DMA sobre fusiones y adquisiciones están destinadas a prevenir que los guardianes consoliden su dominio a través de adquisiciones estratégicas y a asegurar que la competencia siga siendo robusta en los mercados digitales."

},

# Question 16 from DSA

{

    'law': 'dsa',

    'question': "¿Cómo maneja la DSA la cuestión del acoso y abuso en línea?",

    'answer': "La DSA requiere que las plataformas implementen medidas para combatir el acoso y abuso en línea, incluyendo proporcionar a los usuarios herramientas para informar y bloquear contenido y comportamientos abusivos. Las plataformas deben actuar rápidamente para eliminar o deshabilitar el acceso al contenido que constituye acoso o abuso, y deben proporcionar apoyo a las víctimas. La DSA también alienta a las plataformas a colaborar con las autoridades y organizaciones de la sociedad civil para abordar el acoso en línea y desarrollar mejores prácticas para crear un entorno en línea seguro. Estas medidas están destinadas a proteger a los usuarios del daño y promover un espacio digital respetuoso e inclusivo."

},

# Question 17 from GDPR

{

    'law': 'gdpr',

    'question': "¿Cómo aborda el RGPD el tema del consentimiento en el tratamiento de datos?",

    'answer': "Según el RGPD, el consentimiento debe ser libre, específico, informado e inequívoco. Las organizaciones deben asegurarse de que el consentimiento se obtenga mediante una acción afirmativa clara, como marcar una casilla en un sitio web, y que sea distinguible de otros asuntos. Se debe informar al interesado de su derecho a retirar el consentimiento en cualquier momento, y la retirada debe ser tan fácil como otorgar el consentimiento. Además, para los niños menores de 16 años, se requiere el consentimiento parental para el tratamiento de sus datos."

},

# Question 17 from AI Act

{

    'law': 'ai_act',

    'question': "¿Cuál es el papel del Comité Europeo de Inteligencia Artificial (EAIB) bajo la Ley de IA?",

    'answer': "El Comité Europeo de Inteligencia Artificial (EAIB) se establece bajo la Ley de IA para facilitar la cooperación y coordinación entre las autoridades de supervisión nacionales y la Comisión Europea. El EAIB es responsable de emitir directrices, recomendaciones y mejores prácticas sobre la implementación de la Ley de IA, proporcionar asesoramiento a la Comisión Europea sobre cuestiones relacionadas con la IA y promover la aplicación armonizada de la Ley en toda la UE. El EAIB también juega un papel en la resolución de disputas entre autoridades nacionales y en asegurar la consistencia en la interpretación y aplicación de la Ley de IA."

},

# Question 17 from DMA

{

    'law': 'dma',

    'question': "¿Cómo aborda la DMA la cuestión de los patrones oscuros y las prácticas de diseño engañoso por parte de los guardianes?",

    'answer': "La DMA prohíbe a los guardianes usar patrones oscuros y prácticas de diseño engañoso que manipulen o engañen a los usuarios para que tomen decisiones que no son en su mejor interés. Esto incluye prácticas como ocultar información importante, dificultar que los usuarios ejerzan sus derechos o empujar a los usuarios hacia ciertas elecciones. La DMA requiere que los guardianes proporcionen información clara y accesible a los usuarios y que diseñen sus interfaces de manera que respeten la autonomía y la elección del usuario. Estas disposiciones están destinadas a proteger a los consumidores de prácticas manipulativas y asegurar que los servicios digitales sean transparentes y fáciles de usar."

},

# Question 17 from DSA

{

    'law': 'dsa',

    'question': "¿Cómo asegura la DSA que los usuarios tengan control sobre sus datos y privacidad?",

    'answer': "La DSA mejora el control de los usuarios sobre los datos y la privacidad al requerir que las plataformas proporcionen información clara y accesible sobre cómo se recopilan, procesan y utilizan los datos de los usuarios. Los usuarios deben ser informados sobre sus derechos para acceder, rectificar y eliminar sus datos, así como su derecho a oponerse al procesamiento de datos. La DSA también requiere que las plataformas implementen principios de privacidad por diseño y por defecto, asegurando que la privacidad de los usuarios esté protegida desde el principio. Además, las plataformas deben proporcionar a los usuarios herramientas para gestionar sus configuraciones de privacidad y controlar el uso de sus datos para publicidad dirigida."

},

# Question 18 from GDPR

{

    'law': 'gdpr',

    'question': "¿Cuál es el enfoque del RGPD para las transferencias internacionales de datos?",

    'answer': "El RGPD permite las transferencias internacionales de datos solo si el tercer país, territorio u organización internacional asegura un nivel adecuado de protección de datos, según lo determine la Comisión Europea. En ausencia de una decisión de adecuación, las transferencias están permitidas bajo garantías adecuadas, como normas corporativas vinculantes o cláusulas contractuales tipo. En circunstancias específicas, las derogaciones para situaciones específicas, como el consentimiento explícito del interesado, pueden permitir las transferencias. El RGPD busca asegurar que los datos personales transferidos fuera de la UE reciban el mismo nivel de protección que dentro de la UE."

},

# Question 18 from AI Act

{

    'law': 'ai_act',

    'question': "¿Cómo impacta la Ley de IA el uso de la IA en la salud?",

    'answer': "La Ley de IA reconoce los beneficios potenciales de la IA en la salud, como mejorar el diagnóstico, el tratamiento y los resultados de los pacientes. Sin embargo, también reconoce los riesgos asociados con el uso de la IA en este sector sensible. Los sistemas de IA utilizados en salud, particularmente aquellos que involucran la toma de decisiones o proporcionan recomendaciones a profesionales de la salud, se clasifican como de alto riesgo y están sujetos a requisitos estrictos. Estos incluyen garantizar la precisión y la fiabilidad de los sistemas de IA, mantener la supervisión humana y proteger los datos de los pacientes. La Ley también enfatiza la importancia de la transparencia y el consentimiento informado en el uso de la IA en salud."

},

# Question 18 from DMA

{

    'law': 'dma',

    'question': "¿Cómo promueve la DMA la transparencia en la publicidad digital?",

    'answer': "La DMA promueve la transparencia en la publicidad digital al requerir que los guardianes proporcionen a los anunciantes y editores acceso a los datos relacionados con sus campañas publicitarias, incluyendo información sobre precios, rendimiento y criterios de segmentación. Los guardianes también deben asegurar que sus servicios de publicidad se ofrezcan en términos justos, razonables y no discriminatorios, y tienen prohibido usar datos no públicos para obtener una ventaja injusta en el mercado publicitario. Estas disposiciones están destinadas a promover la competencia y la transparencia en la publicidad digital, asegurando que los anunciantes y editores tengan la información que necesitan para tomar decisiones informadas."

},

# Question 18 from DSA

{

    'law': 'dsa',

    'question': "¿Cómo aborda la DSA la cuestión de la transparencia y la responsabilidad algorítmica?",

    'answer': "La DSA requiere que las plataformas, particularmente las VLOPs, proporcionen transparencia sobre cómo funcionan sus algoritmos, incluyendo los criterios utilizados para la recomendación, clasificación y eliminación de contenido. Las plataformas deben explicar la lógica detrás de sus algoritmos y proporcionar a los usuarios opciones para controlar cómo los algoritmos afectan su experiencia en línea. La DSA también obliga a las plataformas a realizar auditorías regulares de sus algoritmos para evaluar su impacto en los usuarios y la sociedad. Estas auditorías deben ser realizadas por terceros independientes y deben evaluar si los algoritmos son justos, no discriminatorios y alineados con los derechos fundamentales."

},

# Question 19 from GDPR

{

    'law': 'gdpr',

    'question': "¿Cómo impacta el RGPD a las pequeñas y medianas empresas (PYMEs)?",

    'answer': "El RGPD reconoce los desafíos a los que se enfrentan las pequeñas y medianas empresas (PYMEs) y proporciona ciertas exenciones y derogaciones para aliviar su carga de cumplimiento. Por ejemplo, las PYMEs con menos de 250 empleados no están obligadas a mantener registros de las actividades de tratamiento a menos que el tratamiento sea regular, suponga un riesgo para los derechos y libertades de los interesados, o implique categorías especiales de datos. Sin embargo, las PYMEs deben seguir cumpliendo con otros requisitos del RGPD, como la notificación de violaciones de datos, los derechos de los interesados y la designación de un DPD si es necesario."

},

# Question 19 from AI Act

{

    'law': 'ai_act',

    'question': "¿Cómo aborda la Ley de IA el tema de la alfabetización en IA y la conciencia pública?",

    'answer': "La Ley de IA fomenta iniciativas para promover la alfabetización en IA y la conciencia pública, reconociendo que los ciudadanos informados y educados son esenciales para la adopción responsable de las tecnologías de IA. La Ley solicita el desarrollo de programas educativos y recursos para ayudar a las personas a comprender las capacidades, limitaciones y riesgos asociados con la IA. También promueve consultas públicas y la participación de partes interesadas para asegurar que las perspectivas de varios grupos, incluidas las de la sociedad civil, se consideren en el desarrollo y despliegue de sistemas de IA."

},

# Question 19 from DMA

{

    'law': 'dma',

    'question': "¿Cómo aborda la DMA la cuestión del acceso a los servicios de plataforma central por parte de los usuarios finales?",

    'answer': "La DMA asegura que los usuarios finales tengan acceso a los servicios de plataforma central en términos justos y no discriminatorios. Los guardianes tienen prohibido restringir o degradar la calidad del acceso a sus servicios o participar en prácticas que limiten la elección del usuario, como obligar a los usuarios a instalar ciertas aplicaciones o usar servicios específicos. La DMA también promueve la portabilidad de datos, permitiendo a los usuarios finales transferir sus datos a otros servicios y aprovechar ofertas competitivas. Estas disposiciones están diseñadas para mejorar la elección y el control del usuario sobre los servicios digitales que utilizan."

},

# Question 19 from DSA

{

    'law': 'dsa',

    'question': "¿Cuáles son los requisitos para que las plataformas en línea cooperen con las autoridades reguladoras bajo la DSA?",

    'answer': "La DSA requiere que las plataformas en línea cooperen con las autoridades reguladoras proporcionando acceso a datos, registros e información necesarios para fines de monitoreo y cumplimiento. Las plataformas deben responder rápidamente a las solicitudes de las autoridades y facilitar inspecciones e investigaciones. La DSA también obliga a las plataformas a proporcionar informes de transparencia y someterse a auditorías independientes para demostrar el cumplimiento con la regulación. La cooperación con las autoridades es esencial para asegurar que las plataformas cumplan con sus obligaciones y que las disposiciones de la DSA se apliquen efectivamente."

},

# Question 20 from GDPR

{

    'law': 'gdpr',

    'question': "¿Qué medidas recomienda el RGPD para garantizar la seguridad de los datos?",

    'answer': "El RGPD requiere que las organizaciones implementen medidas técnicas y organizativas adecuadas para garantizar un nivel de seguridad acorde con el riesgo. Esto incluye medidas como la seudonimización y el cifrado de datos personales, garantizar la confidencialidad, integridad, disponibilidad y resiliencia continuas de los sistemas y servicios de tratamiento, y probar, evaluar y evaluar regularmente la efectividad de las medidas de seguridad. El RGPD también enfatiza la necesidad de realizar evaluaciones de impacto de protección de datos (DPIA) cuando las operaciones de tratamiento probablemente resulten en un alto riesgo para los derechos y libertades de las personas físicas."

},

# Question 20 from AI Act

{

    'law': 'ai_act',

    'question': "¿Qué medidas incluye la Ley de IA para apoyar el desarrollo ético de la IA?",

    'answer': "La Ley de IA apoya el desarrollo ético de la IA fomentando la adopción de códigos de conducta voluntarios, promoviendo la investigación sobre IA ética y promoviendo el desarrollo de sistemas de IA que se alineen con los valores europeos y los derechos fundamentales. La Ley enfatiza la importancia de la IA centrada en el ser humano, donde los sistemas de IA están diseñados para mejorar las capacidades y el bienestar humanos mientras respetan la dignidad y la autonomía humanas. También apoya la creación de entornos regulatorios de prueba (regulatory sandboxes) para permitir a los desarrolladores experimentar con soluciones innovadoras de IA en un entorno controlado, asegurando que las consideraciones éticas se integren en el diseño y despliegue de las tecnologías de IA."

},

# Question 20 from DMA

{

    'law': 'dma',

    'question': "¿Qué papel desempeña la Comisión Europea en la aplicación de la DMA?",

    'answer': "La Comisión Europea es responsable de la aplicación de la DMA, incluida la supervisión del cumplimiento, la realización de investigaciones y la imposición de sanciones por incumplimiento. La Comisión tiene la autoridad para imponer multas, pagos de multas periódicas y remedios estructurales a los guardianes que violen las obligaciones y prohibiciones de la DMA. La Comisión también tiene el poder de iniciar investigaciones de mercado para evaluar si nuevos servicios deben ser designados como servicios de plataforma central o si deben imponerse obligaciones adicionales a los guardianes. La aplicación de la DMA está diseñada para ser robusta y efectiva, asegurando que los guardianes operen de manera que promuevan la competencia y la innovación en los mercados digitales."

},

# Question 20 from DSA

{

    'law': 'dsa',

    'question': "¿Cómo promueve la DSA el desarrollo de códigos de conducta para las plataformas en línea?",

    'answer': "La DSA fomenta el desarrollo de códigos de conducta para las plataformas en línea para abordar cuestiones específicas como la moderación de contenido, la transparencia algorítmica y la protección de menores. Estos códigos de conducta se desarrollan en colaboración con las partes interesadas de la industria, organizaciones de la sociedad civil y autoridades reguladoras. La DSA promueve la adopción de estas medidas voluntarias para asegurar que las plataformas operen de manera responsable y ética. Los códigos de conducta proporcionan un marco para las mejores prácticas y ayudan a las plataformas a alinear sus operaciones con los objetivos de la DSA, mientras permiten flexibilidad e innovación."

}

]





 # Update the laws_info dictionary for GDPR, AI Act, DMA, and DSA

laws_info = {

    'gdpr': {

        'file_path': '/kaggle/input/spanish-laws/spanish_gdpr.html',

        'collection_name': 'embeddings_gdpr',

        'questions_answers': [qa for qa in integrated_questions_answers if qa['law'] == 'gdpr']

    },

    'ai_act': {

        'file_path': '/kaggle/input/spanish-laws/spanish_AI_act.html',

        'collection_name': 'embeddings_ai_act',

        'questions_answers': [qa for qa in integrated_questions_answers if qa['law'] == 'ai_act']

    },

    'dma': {

        'file_path': '/kaggle/input/spanish-laws/spanish_dma.html',

        'collection_name': 'embeddings_dma',

        'questions_answers': [qa for qa in integrated_questions_answers if qa['law'] == 'dma']

    },

    'dsa': {

        'file_path': '/kaggle/input/spanish-laws/spanish_dsa.html',

        'collection_name': 'embeddings_dsa',

        'questions_answers': [qa for qa in integrated_questions_answers if qa['law'] == 'dsa']

    },

}

summarizer = pipeline("summarization", model="philschmid/bart-large-cnn-samsum",device=0)

def summarize_text_huggingface_with_retry(text, max_length=350, min_length=100, max_retries=3):
    for attempt in range(max_retries):
        try:
            # Generate summary
            summary = summarizer(text, max_length=max_length, min_length=min_length, do_sample=False)
            return summary[0]['summary_text']
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt < max_retries - 1:
                print("Retrying...")
                time.sleep(2 ** attempt)  # Exponential backoff
            else:
                print("Max retries reached. Moving to next text.")
                return None


# Function to handle multiple queries and print results for all laws

def embed_and_query_all_laws(laws_info, model_norm, collections, top_k=1):
    similarities = {law: {'cosine': [], 'semantic': []} for law in laws_info}
    for law, info in laws_info.items():

        print(f"\nQuerying {law.upper()} collection:")



        for qa in info['questions_answers']:

            query = qa['question']

            reference_answer = qa['answer']



            # Embed and query

            results = embed_and_query(query, model_norm, collections[law], top_k)



            if results and 'documents' in results and results['documents']:

                retrieved_text = results['documents'][0][0]  # Assuming it's a list of lists
                summary = summarize_text_huggingface_with_retry(retrieved_text)
                
                if summary:
                    qa['summary'] = summary  # Storing the summary in 'qa' for later use
                else:
                    qa['summary'] = None


                # Generate embeddings using BERT for cosine similarity

                retrieved_embedding = generate_bert_embedding(summary, tokenizer, model)

                reference_embedding = generate_bert_embedding(reference_answer, tokenizer, model)



                # Calculate cosine similarity using BERT embeddings

                cosine_sim = calculate_cosine_similarity(reference_embedding, retrieved_embedding)



                # Calculate semantic similarity using Sentence-Transformers model

                semantic_sim = calculate_semantic_similarity(reference_answer, retrieved_text, semantic_model)

                similarities[law]['cosine'].append(cosine_sim)
                similarities[law]['semantic'].append(semantic_sim)


                # Print results

                chunk_id = results['ids'][0][0]  # Accessing the first element in the list of IDs

                print(f"Query: {query}")

                print(f"Retrieved chunk {chunk_id.split('_')[-1]} from {law.upper()}:")

                print(f"Retrieved text: {retrieved_text}")

                print(f"Summary for {law.upper()} - Question: {query}:\n{summary}\n----\n")
                
                print(f"Reference answer: {reference_answer}")

                print(f"Cosine Similarity: {cosine_sim:.4f}")

                print(f"Semantic Similarity: {semantic_sim:.4f}")

                print("----\n")

            else:

                print(f"No valid results found for query: {query} in {law.upper()}.")

    print("\nCalculated Averages:")
    for law in similarities:
        cosine_values = similarities[law]['cosine']
        semantic_values = similarities[law]['semantic']
    
        # Calculate average cosine similarity if there are values
        if cosine_values:
            avg_cosine = sum(cosine_values) / len(cosine_values)
        else:
            avg_cosine = None
    
        # Calculate average semantic similarity if there are values
        if semantic_values:
            avg_semantic = sum(semantic_values) / len(semantic_values)
        else:
            avg_semantic = None
    
        # Print the results
        if avg_cosine is not None and avg_semantic is not None:
            print(f"{law.upper()} Average Cosine Similarity: {avg_cosine:.4f}")
            print(f"{law.upper()} Average Semantic Similarity: {avg_semantic:.4f}")
        else:
            print(f"No valid similarities found for {law.upper()}")
    
# Query and print results for all questions across GDPR, AI Act, DMA, and DSA

embed_and_query_all_laws(laws_info, model_norm, collections, top_k=1)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Your max_length is set to 350, but your input_length is only 243. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=121)



Querying GDPR collection:


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: ¿Cuál es el derecho fundamental en relación con el tratamiento de datos personales según la Carta de los Derechos Fundamentales de la Unión Europea?
Retrieved chunk 49 from GDPR:
Retrieved text: Dichas restricciones deben ajustarse a lo dispuesto en la Carta y en el Convenio Europeo para la Protección de los Derechos Humanos y de las Libertades Fundamentales. (74) Debe quedar establecida la responsabilidad del responsable del tratamiento por cualquier tratamiento de datos personales realizado por él mismo o por su cuenta. En particular, el responsable debe estar obligado a aplicar medidas oportunas y eficaces y ha de poder demostrar la conformidad de las actividades de tratamiento con el presente Reglamento, incluida la eficacia de las medidas. Dichas medidas deben tener en cuenta la naturaleza, el ámbito, el contexto y los fines del tratamiento así como el riesgo para los derechos y libertades de las personas físicas.
Summary for GDPR - Question: ¿Cuál es el derecho fundamental

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: ¿Cuáles son los requisitos para que las plataformas en línea cooperen con las autoridades reguladoras bajo la DSA?
Retrieved chunk 73 from DSA:
Retrieved text: Además, y después de permitir la oferta del producto o servicio por parte del comerciante, los prestadores de plataformas en línea de que se trate deben hacer todos los esfuerzos que resulten razonables para comprobar aleatoriamente si los productos o servicios ofrecidos han sido identificados como ilícitos en cualquier base de datos en línea o interfaz en línea oficial, de libre acceso y legible por máquina disponible en un Estado miembro o en la Unión. La Comisión también debe fomentar la trazabilidad de los productos a través de soluciones tecnológicas, como códigos de respuesta rápida firmados digitalmente (o «códigos QR») o fichas no fungibles. La Comisión debe promover el desarrollo de normas y, a falta de ellas, soluciones basadas en el mercado que puedan ser aceptables para las partes afectadas. (75) Dada la impor

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: ¿Cómo promueve la DSA el desarrollo de códigos de conducta para las plataformas en línea?
Retrieved chunk 106 from DSA:
Retrieved text: (106) Las disposiciones del presente Reglamento relativas a los códigos de conducta podrían servir de base para iniciativas de autorregulación ya establecidas a escala de la Unión, como el compromiso de seguridad con los productos, el memorando de entendimiento sobre la venta de productos falsificados a través de internet, el Código de conducta para la lucha contra la incitación ilegal al odio en internet, y el Código de buenas prácticas en materia de desinformación. En particular en relación con este último, a raíz de las orientaciones de la Comisión, el Código de buenas prácticas en materia de desinformación ha sido reforzado como se anunció en el Plan de Acción para la Democracia Europea. (107) En la publicidad en línea generalmente intervienen varios actores, incluidos los servicios intermediarios que conectan a los publicistas con los anunc

In [13]:
from collections import defaultdict

from transformers import pipeline



# Load the Hugging Face su mmarization model

summarizer = pipeline("summarization", model="philschmid/bart-large-cnn-samsum",device=0)



# Convert integrated questions and answers to a dictionary format

def create_laws_info(questions_answers_list):

    """

    Dynamically creates a dictionary of laws and their questions and answers.



    Args:

    - questions_answers_list (list): List of dictionaries containing law, question, and answer data.



    Returns:

    - laws_info (dict): A dictionary with each law as a key, and associated questions and answers.

    """

    laws_info = defaultdict(lambda: {'questions_answers': []})



    # Group questions and answers by law

    for entry in questions_answers_list:

        law = entry['law']

        laws_info[law]['questions_answers'].append(entry)



    return laws_info



# Automatically construct laws_info dictionary

laws_info = create_laws_info(integrated_questions_answers)



# Function to summarize a given text using Hugging Face model with retry logic

def summarize_text_huggingface_with_retry(text, max_length=350, min_length=100, max_retries=3):

    for attempt in range(max_retries):

        try:

            # Generate summary

            summary = summarizer(text, max_length=max_length, min_length=min_length, do_sample=False)

            return summary[0]['summary_text']

        except Exception as e:

            print(f"Attempt {attempt + 1} failed: {e}")

            if attempt < max_retries - 1:

                print("Retrying...")

                time.sleep(2 ** attempt)  # Exponential backoff

            else:

                print("Max retries reached. Moving to next text.")

                return None



# Function to handle summarization of chunks for all laws using Hugging Face model

def summarize_all_laws_huggingface(laws_info):

    for law, info in laws_info.items():

        print(f"\nProcessing summaries for {law.upper()}...")



        for qa in info['questions_answers']:

            question = qa['question']

            retrieved_text = qa['answer']



            print(f"Processing {law.upper()} - Question: {question}")



            # Generate a summary for each retrieved text using the Hugging Face model

            summary = summarize_text_huggingface_with_retry(retrieved_text)

            if summary:

                qa['summary'] = summary  # Save the summary in the dictionary

                print(f"Summary for {law.upper()} - Question: {question}:\n{summary}\n----\n")

            else:

                print(f"Failed to get summary for {law.upper()} - Question: {question}\n")



# Run the summarization for all laws

summarize_all_laws_huggingface(laws_info)


config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/300 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Summary for DSA - Question: ¿Cómo asegura la DSA que los usuarios tengan control sobre sus datos y privacidad?:
La DSA mejora el control de los usuarios sobre los datos y la privacidad. La DSA requiere that las plataformas proporcionen información clara y accesible sobre cómo se recopilan, procesan and utilize datos de usuario. La dsa también requiere plataforms implement principios de privacidios de diseño and por defecto.
----

Processing DSA - Question: ¿Cómo aborda la DSA la cuestión de la transparencia y la responsabilidad algorítmica?


Your max_length is set to 350, but your input_length is only 208. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=104)


Summary for DSA - Question: ¿Cómo aborda la DSA la cuestión de la transparencia y la responsabilidad algorítmica?:
La DSA requires las plataformas to proporcionen transparencia sobre cómo funcionan sus algoritmos. La DSA also requires auditorías regulares de sus algorithms sobre their impacto on usuarios and la sociedad. Esta auditoría deben evaluarse if the algorithms are justos, discriminatorios, alineados with los derechos fundamentales, and justos. 
----

Processing DSA - Question: ¿Cuáles son los requisitos para que las plataformas en línea cooperen con las autoridades reguladoras bajo la DSA?


Your max_length is set to 350, but your input_length is only 234. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=117)


Summary for DSA - Question: ¿Cuáles son los requisitos para que las plataformas en línea cooperen con las autoridades reguladoras bajo la DSA?:
La DSA requiere que las plataformas cooperen con las autoridades reguladoras. La cooperación is esencial para asegurar que las Plataforma cumplan con sus obligaciones. La DSA también obliga a las plATAformas a proporcionar informes de transparencia and someterse to auditorías independentes.      Â
----

Processing DSA - Question: ¿Cómo promueve la DSA el desarrollo de códigos de conducta para las plataformas en línea?
Summary for DSA - Question: ¿Cómo promueve la DSA el desarrollo de códigos de conducta para las plataformas en línea?:
La DSA fomenta el desarrollo de códigos de conducta para las plataformas en línea. Estos códicos of conducta proporcionan un marco para las mejores prácticas. La DSA promueve la adopción de estas medidas to asegurar que las plATAformas operen de manera responsable and ética.
----



**compare the summaries with the reference answers and calculates the cosine and semantic similarity of each and their averages**


In [14]:
# Function to compare summaries with reference answers and calculate averages

def compare_summaries_with_reference_and_calculate_averages(laws_info, tokenizer, model, semantic_model):

    # Initialize dictionaries to store cosine and semantic similarities for each law

    similarities = {law: {'cosine': [], 'semantic': []} for law in laws_info}



    for law, info in laws_info.items():

        print(f"\nProcessing law: {law.upper()}")



        for qa in info['questions_answers']:

            query = qa['question']

            reference_answer = qa['answer']

            summary = qa.get('summary')  # Access the stored summary



            if summary:

                # Step 1: Generate embeddings for cosine similarity

                reference_embedding = generate_bert_embedding(reference_answer, tokenizer, model)

                summary_embedding = generate_bert_embedding(summary, tokenizer, model)



                # Step 2: Calculate cosine similarity

                cosine_sim = calculate_cosine_similarity(reference_embedding, summary_embedding)



                # Step 3: Calculate semantic similarity

                semantic_sim = calculate_semantic_similarity(reference_answer, summary, semantic_model)



                # Step 4: Store the cosine and semantic similarities

                similarities[law]['cosine'].append(cosine_sim)

                similarities[law]['semantic'].append(semantic_sim)



                # Step 5: Print the individual results

                print(f"Query: {query}")

                print(f"Summary: {summary}")

                print(f"Reference answer: {reference_answer}")

                print(f"Cosine Similarity: {cosine_sim:.4f}")

                print(f"Semantic Similarity: {semantic_sim:.4f}")

                print("----\n")

            else:

                print(f"No summary available for query: {query} in {law.upper()}")



    # Calculate and print the averages

    print("\nCalculated Averages:")

    for law in similarities:

        cosine_values = similarities[law]['cosine']

        semantic_values = similarities[law]['semantic']



        # Calculate average cosine similarity if there are values

        avg_cosine = sum(cosine_values) / len(cosine_values) if cosine_values else None

        # Calculate average semantic similarity if there are values

        avg_semantic = sum(semantic_values) / len(semantic_values) if semantic_values else None



        # Print the results

        if avg_cosine is not None and avg_semantic is not None:

            print(f"{law.upper()} Average Cosine Similarity: {avg_cosine:.4f}")

            print(f"{law.upper()} Average Semantic Similarity: {avg_semantic:.4f}")

        else:

            print(f"No valid similarities found for {law.upper()}")



# Run the comparison function

compare_summaries_with_reference_and_calculate_averages(laws_info, tokenizer, model, semantic_model)



Processing law: GDPR


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: ¿Cuál es el derecho fundamental en relación con el tratamiento de datos personales según la Carta de los Derechos Fundamentales de la Unión Europea?
Summary: La protección de datos personales es un derecho fundamental. La Carta de los Derechos Fundamentales de la Unión Europea (‘la Carta’) establecen toda personas tiene derecho a la proteccion de los datos that le concierne. Este Reglamento pretende contribuir a la realización de un área de libertad, seguridad, justicia, económica, progreso econociero and social.
Reference answer: La protección de las personas físicas en relación con el tratamiento de datos personales es un derecho fundamental. El artículo 8(1) de la Carta de los Derechos Fundamentales de la Unión Europea (‘la Carta’) y el artículo 16(1) del Tratado de Funcionamiento de la Unión Europea (TFUE) establecen que toda persona tiene derecho a la protección de los datos personales que le conciernen. Este Reglamento pretende contribuir a la realización de un área de lib

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: ¿Cuáles son los requisitos para que las plataformas en línea cooperen con las autoridades reguladoras bajo la DSA?
Summary: La DSA requiere que las plataformas cooperen con las autoridades reguladoras. La cooperación is esencial para asegurar que las Plataforma cumplan con sus obligaciones. La DSA también obliga a las plATAformas a proporcionar informes de transparencia and someterse to auditorías independentes.      Â
Reference answer: La DSA requiere que las plataformas en línea cooperen con las autoridades reguladoras proporcionando acceso a datos, registros e información necesarios para fines de monitoreo y cumplimiento. Las plataformas deben responder rápidamente a las solicitudes de las autoridades y facilitar inspecciones e investigaciones. La DSA también obliga a las plataformas a proporcionar informes de transparencia y someterse a auditorías independientes para demostrar el cumplimiento con la regulación. La cooperación con las autoridades es esencial para asegurar que

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: ¿Cómo promueve la DSA el desarrollo de códigos de conducta para las plataformas en línea?
Summary: La DSA fomenta el desarrollo de códigos de conducta para las plataformas en línea. Estos códicos of conducta proporcionan un marco para las mejores prácticas. La DSA promueve la adopción de estas medidas to asegurar que las plATAformas operen de manera responsable and ética.
Reference answer: La DSA fomenta el desarrollo de códigos de conducta para las plataformas en línea para abordar cuestiones específicas como la moderación de contenido, la transparencia algorítmica y la protección de menores. Estos códigos de conducta se desarrollan en colaboración con las partes interesadas de la industria, organizaciones de la sociedad civil y autoridades reguladoras. La DSA promueve la adopción de estas medidas voluntarias para asegurar que las plataformas operen de manera responsable y ética. Los códigos de conducta proporcionan un marco para las mejores prácticas y ayudan a las plataformas

# calculates the averages of the cosine and semanic similarity comparison of the retrieved text and the reference answer

In [15]:
from IPython.display import display

from IPython.utils.io import capture_output



# Suppress unwanted output while calculating similarities

with capture_output() as captured:

    # Initialize dictionaries to store similarities for each law

    similarities = {

        'gdpr': {'cosine': [], 'semantic': []},

        'ai_act': {'cosine': [], 'semantic': []},

        'dma': {'cosine': [], 'semantic': []},

        'dsa': {'cosine': [], 'semantic': []}

    }



    for law, info in laws_info.items():

        for qa in info['questions_answers']:

            query = qa['question']

            reference_answer = qa['answer']



            # Embed and query

            results = embed_and_query(query, model_norm, collections[law], top_k=1)



            if results and 'documents' in results and results['documents']:

                retrieved_text = results['documents'][0][0]



                # Generate embeddings using BERT for cosine similarity

                retrieved_embedding = generate_bert_embedding(retrieved_text, tokenizer, model)

                reference_embedding = generate_bert_embedding(reference_answer, tokenizer, model)



                # Calculate similarities

                cosine_sim = calculate_cosine_similarity(reference_embedding, retrieved_embedding)

                semantic_sim = calculate_semantic_similarity(reference_answer, retrieved_text, semantic_model)



                # Store in the appropriate law's list

                similarities[law]['cosine'].append(cosine_sim)

                similarities[law]['semantic'].append(semantic_sim)



# Calculate and print averages for each law

print("Calculated Averages:")

for law in similarities:

    avg_cosine = sum(similarities[law]['cosine']) / len(similarities[law]['cosine'])

    avg_semantic = sum(similarities[law]['semantic']) / len(similarities[law]['semantic'])



    print(f"{law.upper()} Average Cosine Similarity: {avg_cosine:.4f}")

    print(f"{law.upper()} Average Semantic Similarity: {avg_semantic:.4f}")


Calculated Averages:
GDPR Average Cosine Similarity: 0.8136
GDPR Average Semantic Similarity: 0.9991
AI_ACT Average Cosine Similarity: 0.7464
AI_ACT Average Semantic Similarity: 0.9992
DMA Average Cosine Similarity: 0.7145
DMA Average Semantic Similarity: 0.9991
DSA Average Cosine Similarity: 0.7973
DSA Average Semantic Similarity: 0.9993


**calculates the latency of the retrieved chunks**

In [16]:
import time



# Function to measure query latency and calculate averages

def measure_and_average_query_latency(laws_info, model_norm, collections, top_k=1):

    latency_results = {

        'gdpr': [],

        'ai_act': [],

        'dma': [],

        'dsa': []

    }

    all_latencies = []



    for law, info in laws_info.items():

        print(f"\nMeasuring query latency for {law.upper()} collection:")



        for qa in info['questions_answers']:

            query = qa['question']



            # Record start time

            start_time = time.time()



            # Embed and query

            results = embed_and_query(query, model_norm, collections[law], top_k)



            # Record end time

            end_time = time.time()



            # Calculate latency

            latency = end_time - start_time

            latency_results[law].append(latency)

            all_latencies.append(latency)



            print(f"Query: {query}")

            print(f"Latency: {latency:.4f} seconds")

            print("----\n")



    # Calculate and print average latency for each law

    for law in latency_results:

        if latency_results[law]:  # Check if the list is not empty

            avg_latency = sum(latency_results[law]) / len(latency_results[law])

            print(f"{law.upper()} Average Query Latency: {avg_latency:.4f} seconds")

        else:

            print(f"{law.upper()} has no recorded latencies.")



    # Calculate and print the overall average latency across all laws

    if all_latencies:

        overall_avg_latency = sum(all_latencies) / len(all_latencies)

        print(f"\nOverall Average Query Latency: {overall_avg_latency:.4f} seconds")

    else:

        print("No latencies recorded across all laws.")



# Run the latency measurement and averaging function

measure_and_average_query_latency(laws_info, model_norm, collections, top_k=1)



Measuring query latency for GDPR collection:
Query: ¿Cuál es el derecho fundamental en relación con el tratamiento de datos personales según la Carta de los Derechos Fundamentales de la Unión Europea?
Latency: 0.0208 seconds
----

Query: ¿Cómo pretende el RGPD equilibrar el derecho a la protección de datos personales con otros derechos fundamentales?
Latency: 0.0164 seconds
----

Query: ¿Qué desafíos han surgido debido a los desarrollos tecnológicos y la globalización en el contexto de la protección de datos personales?
Latency: 0.0159 seconds
----

Query: ¿Cómo aborda el RGPD la transferencia de datos personales a terceros países u organizaciones internacionales?
Latency: 0.0154 seconds
----

Query: ¿Qué protecciones específicas ofrece el RGPD a los niños con respecto a sus datos personales?
Latency: 0.0161 seconds
----

Query: ¿Cómo define el RGPD los datos personales y cuáles son algunos ejemplos?
Latency: 0.0159 seconds
----

Query: ¿Cuál es la base legal para el tratamiento de da

# 80


In [11]:
import pandas as pd



# Function to load and inspect the CSV

def inspect_csv(file_path):

    # Read the CSV file with quotechar to handle commas within the fields

    df = pd.read_csv(file_path, encoding='utf-8', delimiter="/,", quotechar='"', skip_blank_lines=True)



    # Print column names to see the headers

    print("Column Names:", df.columns)



    # Print the first 5 rows of the data to inspect the format

    print("\nFirst 5 Rows:\n", df.head())



    # Optionally, print the data types of each column

    print("\nData Types:\n", df.dtypes)



# Example: inspecting a CSV file

gdpr_csv_path = '/kaggle/input/spanish-80-qa/q and a gdpr 80 spanish.csv'

inspect_csv(gdpr_csv_path)


Column Names: Index(['Pregunta: Puede un miembro de una autoridad de control ejercer una actividad remunerada durante su mandato?'], dtype='object')

First 5 Rows:
   Pregunta: Puede un miembro de una autoridad de control ejercer una actividad remunerada durante su mandato?
0                              Respuesta: En general                                                         
1  Pregunta: ¿Cuál es el papel de una autoridad d...                                                         
2  Respuesta: Las autoridades de control son cruc...                                                         
3  Pregunta: ¿Cuáles son las implicaciones jurídi...                                                         
4  Respuesta: La transferencia de datos personale...                                                         

Data Types:
 Pregunta: Puede un miembro de una autoridad de control ejercer una actividad remunerada durante su mandato?    object
dtype: object


/tmp/ipykernel_30/2509536837.py:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(file_path, encoding='utf-8', delimiter="/,", quotechar='"', skip_blank_lines=True)


In [12]:
import pandas as pd

# Function to load and split questions and answers into separate columns

def load_and_split_questions_answers(file_path):

    df = pd.read_csv(file_path, encoding='utf-8',delimiter="/,", header=None)  # Reading without a header

    # Lists to store the questions and answers

    questions = []

    answers = []

    current_question = None  # Temporary storage for the current question

    for index, row in df.iterrows():

        text = row[0]  # Access the single column

        # Ensure that the text is a valid string (skip if NaN)

        if pd.isna(text):

            continue

        text = str(text).lstrip().replace('\xa0', ' ')  # Handling any non-breaking spaces

        # Check if the row is a question

        if text.startswith("Pregunta:"):

            current_question = text.replace("Pregunta: ", "").strip()  # Clean the question text

        elif text.startswith("Respuesta:") and current_question:

            # If it's an answer, append both the question and answer to the lists

            answer = text.replace("Respuesta: ", "").strip()

            questions.append(current_question)

            answers.append(answer)

            current_question = None  # Reset after storing

    # Create a DataFrame with separate columns for questions and answers

    qa_df = pd.DataFrame({'question': questions, 'answer': answers})

    return qa_df

# List of CSV file paths for the different laws

csv_paths = {

    'gdpr': '/kaggle/input/spanish-80-qa/q and a gdpr 80 spanish.csv',

    'ai_act': '/kaggle/input/spanish-80-qa/q and a spanish 80 ai.csv',

    'dma': '/kaggle/input/spanish-80-qa/questions_answers_digital_marketing es.csv',

    'dsa': '/kaggle/input/spanish-80-qa/questions_answers_digital_services es.csv'

}

# Dictionary to store the QA pairs DataFrames for each law

qa_dataframes = {}

# Loop over the CSV paths and process each one

for law, path in csv_paths.items():
    print(path)
    qa_dataframes[law] = load_and_split_questions_answers(path)

    print(f"Processed {law.upper()} law:")

    print(qa_dataframes[law].head(), "\n")

# Now, you have each law's question-answer pairs stored in the `qa_dataframes` dictionary

# You can access the GDPR data, for example, with: qa_dataframes['gdpr']


/kaggle/input/spanish-80-qa/q and a gdpr 80 spanish.csv
Processed GDPR law:
                                            question  \
0  Puede un miembro de una autoridad de control e...   
1  ¿Cuál es el papel de una autoridad de control ...   
2  ¿Cuáles son las implicaciones jurídicas de tra...   
3  ¿Cómo se aplica el principio de tutela judicia...   
4  ¿Qué es el principio de subsidiariedad y cómo ...   

                                              answer  
0                                         En general  
1  Las autoridades de control son cruciales para ...  
2  La transferencia de datos personales a un terc...  
3  El principio de tutela judicial efectiva garan...  
4  El principio de subsidiariedad dicta que la UE...   

/kaggle/input/spanish-80-qa/q and a spanish 80 ai.csv
Processed AI_ACT law:
                                            question  \
0  ¿Cuáles son las implicaciones jurídicas de que...   
1  ¿Puede explicar la interacción entre la protec...   
2  ¿Cuáles 

/tmp/ipykernel_30/2185574145.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(file_path, encoding='utf-8',delimiter="/,", header=None)  # Reading without a header
/tmp/ipykernel_30/2185574145.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(file_path, encoding='utf-8',delimiter="/,", header=None)  # Reading without a header
/tmp/ipykernel_30/2185574145.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning b

In [13]:
import chromadb

from transformers import pipeline

from sentence_transformers import SentenceTransformer

from sklearn.metrics.pairwise import cosine_similarity



# Load Hugging Face summarization model

summarizer = pipeline("summarization", model="philschmid/bart-large-cnn-samsum",device=0)

# Load SBERT model for embedding queries (for cosine similarity)

model_norm = SentenceTransformer('camembert-base')



# Load SBERT model for semantic similarity

semantic_similarity_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')



# Function to retrieve relevant chunks based on the query embedding

def embed_and_query(query_text, model_norm, collection, top_k=1):

    query_embedding = model_norm.encode([query_text])[0]  # Embed the query

    results = collection.query(query_embeddings=[query_embedding], n_results=top_k)

    return results



# Function to calculate cosine similarity between two embeddings

def calculate_cosine_similarity(embedding1, embedding2):

    return cosine_similarity([embedding1], [embedding2])[0][0]



# Function to calculate semantic similarity using a dedicated model

def calculate_semantic_similarity(text1, text2):

    # Use the pre-trained model to compute semantic similarity (returns NumPy embeddings directly)

    embedding1 = semantic_similarity_model.encode(text1)

    embedding2 = semantic_similarity_model.encode(text2)



    # Calculate cosine similarity using sklearn (no conversion needed)

    cosine_sim = cosine_similarity([embedding1], [embedding2])[0][0]

    return cosine_sim



# Function to summarize and compare the retrieved text with the reference answers

def summarize_and_compare_all_laws(qa_dataframes, model_norm, collections):

    similarities = {

        'gdpr': {'cosine': [], 'semantic': []},

        'ai_act': {'cosine': [], 'semantic': []},

        'dma': {'cosine': [], 'semantic': []},

        'dsa': {'cosine': [], 'semantic': []}

    }



    for law, df in qa_dataframes.items():

        print(f"\nProcessing summaries for {law.upper()} law:")



        question_col = 'question'

        answer_col = 'answer'



        for index, row in df.iterrows():

            question_text = row[question_col]  # Get the question from the DataFrame

            answer_text = row[answer_col]  # Get the reference answer



            print(f"Retrieving chunk for query: {question_text}")



            # Step 1: Retrieve the most relevant chunk from ChromaDB

            results = embed_and_query(question_text, model_norm, collections[law], top_k=1)

            if results and 'documents' in results and results['documents']:

                retrieved_chunk = results['documents'][0]  # Get the top retrieved result



                print(f"Summarizing retrieved chunk for query: {question_text}")



                # Step 2: Summarize the retrieved chunk

                summary = summarize_text_huggingface_with_retry(retrieved_chunk[0])  # Summarize the top chunk



                if summary:

                    print(f"Original answer: {answer_text}")

                    print(f"Retrieved chunk: {retrieved_chunk[0]}")

                    print(f"Generated summary: {summary}\n")



                    # Step 3: Compare the summary with the original answer using cosine similarity

                    answer_embedding = model_norm.encode([answer_text])[0]  # Embed reference answer

                    summary_embedding = model_norm.encode([summary])[0]  # Embed generated summary



                    # Calculate cosine similarity

                    cosine_sim = calculate_cosine_similarity(answer_embedding, summary_embedding)



                    # Calculate semantic similarity using the dedicated model

                    semantic_sim = calculate_semantic_similarity(answer_text, summary)



                    # Append the calculated similarities to the lists

                    similarities[law]['cosine'].append(cosine_sim)

                    similarities[law]['semantic'].append(semantic_sim)



                    print(f"Cosine similarity between summary and original answer: {cosine_sim:.4f}")

                    print(f"Semantic similarity between summary and original answer: {semantic_sim:.4f}\n")

                else:

                    print(f"Failed to generate a summary for query: {question_text}\n")

            else:

                print(f"No valid results found for query: {question_text} in {law.upper()} law")



    # Calculate and print averages for each law

    print("Calculated Averages:")

    for law in similarities:

        if similarities[law]['cosine'] and similarities[law]['semantic']:

            avg_cosine = sum(similarities[law]['cosine']) / len(similarities[law]['cosine'])

            avg_semantic = sum(similarities[law]['semantic']) / len(similarities[law]['semantic'])

            print(f"{law.upper()} Average Cosine Similarity: {avg_cosine:.4f}")

            print(f"{law.upper()} Average Semantic Similarity: {avg_semantic:.4f}")

        else:

            print(f"No similarities found for {law.upper()}")



    return similarities



# Store embeddings and chunks in ChromaDB

def process_and_store_embeddings(file_path, collection_name, model_norm):

    chunks = load_and_process_html(file_path)  # Get the text chunks

    embeddings = model_norm.encode(chunks)  # Embed chunks



    chroma_client = chromadb.Client()  # Initialize ChromaDB client

    collection = setup_chroma_collection(chroma_client, collection_name)  # Setup the collection



    for i, embedding in enumerate(embeddings):

        collection.add(

            documents=[chunks[i]],

            ids=[f"id_{i}"],

            embeddings=[embedding]

        )

    return collection



# Process the documents and store embeddings in ChromaDB

gdpr_collection = process_and_store_embeddings('/kaggle/input/spanish-laws/spanish_gdpr.html', 'gdpr_collection', model_norm)

ai_act_collection = process_and_store_embeddings('/kaggle/input/spanish-laws/spanish_AI_act.html', 'ai_act_collection', model_norm)

dma_collection = process_and_store_embeddings('/kaggle/input/spanish-laws/spanish_dma.html', 'dma_collection', model_norm)

dsa_collection = process_and_store_embeddings('/kaggle/input/spanish-laws/spanish_dsa.html', 'dsa_collection', model_norm)



# Now add collections to a dictionary

collections = {

    'gdpr': gdpr_collection,

    'ai_act': ai_act_collection,

    'dma': dma_collection,

    'dsa': dsa_collection

}



# Summarize and compare for all laws

summarize_and_compare_all_laws(qa_dataframes, model_norm, collections);


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Cosine similarity between summary and original answer: 0.9660
Semantic similarity between summary and original answer: 0.3950

Calculated Averages:
GDPR Average Cosine Similarity: 0.9213
GDPR Average Semantic Similarity: 0.3566
AI_ACT Average Cosine Similarity: 0.9291
AI_ACT Average Semantic Similarity: 0.3403
DMA Average Cosine Similarity: 0.9428
DMA Average Semantic Similarity: 0.2783
DSA Average Cosine Similarity: 0.9550
DSA Average Semantic Similarity: 0.3764
